<a href="https://colab.research.google.com/github/Seboldju/OCR-Stromzaehler-Arbeitsdokument/blob/main/Schuster_Project2_YOLOv5_CRNET_Abgabe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Automatic Utility Meter Reading from Photos

Thomas Bowduj (318186);
Niklas Kluge (329596);
Jan Riedel (322148);
Julian Sebold (321768)


# 1. Einführung
Im Rahmen des Projektes wurden verschiedene Ansätze untersucht, um das Projektziel zu erreichen.

  ### 1. Counter Detection (Yolov5) - Jan & Julian
  ### 2. Digit Detection (CR-NET) - Jan & Julian
3. Digit Segmentation / Digit Recognition (CNN) - Niklas
4. Digit Detection / Recognition (OCR) - Thomas

Das Projekt wurde in enger Absprache mit allen Projektmitglieder über die einzelnen Projektphasen durchgeführt und zu gleichen Teilen erledigt.
Für die oben genannten Ansätze war jeweils eine Person im Lead. Aufgrund der Kombination der Ansätze 1. und 2. zum besten Gesamtmodell haben Jan und Julian so eng zusammengearbeitet, sodass hier beide in gleichen Teilen verantwortlich waren.
Aufgrund der Tatsachen, dass unsere zur Verfügung stehender Datensatz sehr groß ist, wir aber nicht auf Daten verzichten wollten, und auch die Modelle selber sehr groß wurden, war es uns nicht möglich das Projekt vollumfänglich in Colab abzubilden. Daher wurde das Projekt hauptsächlich über die Microsoft Python-Erweiterung in VS Code ausgeführt.

### Jedoch wurde dieses Colab von Jan und Julian für eine ausführliche Beschreibung des Gesamtmodells (Kombination YOLOv5 - CRNET) erstellt


In [ ]:
### Installation Bibliotheken

!pip install absl-py==2.1.0 \
albucore==0.0.12 \
albumentations==1.4.11 \
annotated-types==0.7.0 \
astunparse==1.6.3 \
certifi==2024.7.4 \
charset-normalizer==3.3.2 \
cloudpickle==3.0.0 \
contourpy==1.2.1 \
cycler==0.12.1 \
easyocr==1.7.1 \
eval_type_backport==0.2.0 \
filelock==3.15.4 \
flatbuffers==24.3.25 \
fonttools==4.53.1 \
fsspec==2024.6.1 \
future==1.0.0 \
gast==0.6.0 \
gitdb==4.0.11 \
GitPython==3.1.43 \
google-pasta==0.2.0 \
grpcio==1.64.1 \
h5py==3.11.0 \
hyperopt==0.2.7 \
idna==3.7 \
imageio==2.34.2 \
Jinja2==3.1.4 \
joblib==1.4.2 \
keras==3.4.1 \
kiwisolver==1.4.5 \
lazy_loader==0.4 \
libclang==18.1.1 \
Markdown==3.6 \
markdown-it-py==3.0.0 \
MarkupSafe==2.1.5 \
matplotlib==3.9.1 \
mdurl==0.1.2 \
ml-dtypes==0.3.2 \
mpmath==1.3.0 \
namex==0.0.8 \
networkx==3.3 \
ninja==1.11.1.1 \
numpy==1.26.4 \
opencv-python==4.10.0.84 \
opencv-python-headless==4.10.0.84 \
opt-einsum==3.3.0 \
optree==0.12.1 \
packaging==24.1 \
pandas==2.2.2 \
pillow==10.4.0 \
pip==24.1.2 \
protobuf==4.25.3 \
psutil==6.0.0 \
py-cpuinfo==9.0.0 \
py4j==0.10.9.7 \
pyclipper==1.3.0.post5 \
pycocotools==2.0.8 \
pydantic==2.8.2 \
pydantic_core==2.20.1 \
Pygments==2.18.0 \
pyparsing==3.1.2 \
python-bidi==0.4.2 \
python-dateutil==2.9.0.post0 \
pytz==2024.1 \
PyYAML==6.0.1 \
requests==2.32.3 \
rich==13.7.1 \
scikit-image==0.24.0 \
scikit-learn==1.5.1 \
scipy==1.14.0 \
seaborn==0.13.2 \
setuptools==70.3.0 \
shapely==2.0.5 \
six==1.16.0 \
smmap==5.0.1 \
sympy==1.13.0 \
tensorboard==2.16.2 \
tensorboard-data-server==0.7.2 \
tensorflow==2.16.2 \
termcolor==2.4.0 \
thop==0.1.1-2209072238 \
threadpoolctl==3.5.0 \
tifffile==2024.7.2 \
tomli==2.0.1 \
torch==2.3.1 \
torchvision==0.18.1 \
tqdm==4.66.4 \
typing_extensions==4.12.2 \
tzdata==2024.1 \
ultralytics==8.2.55 \
ultralytics-thop==2.0.0 \
urllib3==2.2.2 \
Werkzeug==3.0.3 \
wheel==0.43.0 \
wrapt==1.16.0



#2. Datensatz

Grundlage für das Projekt und Voraussetzung für unser Training war ein geeigneter Datensatz mit Testbildern von Stromzählern. Ursprünglich war eine Kooperation mit den Stadtwerken Pforzheim geplant, die uns aber keine Bilddaten zur Verfügung stellen konnten. Nach Rücksprache mit Prof. Schuster entschieden wir uns für einen Datensatz aus dem Internet von R. Laroca et al. (2019). Dieser entstand im Rahmen der Forschungsarbeit 'Convolutional Neural Networks for Automatic Meter Reading' und wurde uns auf Anfrage per E-Mail über einen Download-Link zur Verfügung gestellt.
Der verwendete „UFPR-AMR“-Datensatz enthält 2000 Bilder von Stromzählern aus einem Lager der Energiegesellschaft von Paraná (Copel), die über 4 Millionen Verbraucher im brasilianischen Bundesstaat Paraná direkt beliefert. Jedem Bild ('image0001.jpg') sind Textdateien ('image0001.txt') zugeordnet, die die Nutzungskamera des Bildes, den Zählerstand des gleichnamigen Bildes sowie die Markierungen für die Positionen der Zählerstände und die einzelnen Ziffern enthalten.

In [ ]:
### Beispielhafte neu erstellte Struktur der Textdatei für das Bild 'meter0001.txt' :

camera: LG G3 D855
reading: 00001
position: 879 1775 434 178
	digit 1: 888 1788 84 146
	digit 2: 978 1788 76 144
	digit 3: 1057 1788 77 143
	digit 4: 1140 1788 77 143
	digit 5: 1229 1792 66 136

# 3. Counter Detection (Ordner "counter detection" im Repository)
Bei der Durchsicht des Datensatzes haben wir festgestellt, dass die Zähler viele Textblöcke haben, die mit der Zähleranzeige verwechselt werden können. Der Zähler, der unsere Region of Interest (ROI) darstellt, nimmt normalerweise nur einen kleinen Teil des Bildes ein. Die Position des Zählers hängt von vielen Faktoren ab, wie z. B. dem Zählertyp, dem Abstand zum Zähler, dem Knöchel und so weiter. Daher zielt unser Ansatz darauf ab, zunächst die Zählerregionen zu extrahieren und das Bild entsprechend dem Zähler zuzuschneiden. Dadurch können die nächsten Schritte, die Zifferndetektion und -erkennung, innerhalb der zuvor identifizierten Zähler stattfinden, was die oben erwähnten Störfaktoren wie Textblöcke deutlich reduzieren sollte.

Für unsere spezifischen Anforderungen haben wir uns entschieden, ein YOLOv5-Modell zu verwenden und es an unseren Datensatz anzupassen. YOLOv5 ist ein leistungsfähiger und effizienter Objektdetektor, ideal für die Echtzeit-Objekterkennung. Somit eignet es sich auch sehr gut für Bilder, die mit dem Smartphone aufgenommen werden.

Das Yolov5-Modell wurde für unseren Projektschritt in Visual Studio Code im Terminal geklont und im Ordner 'detector' abgelegt.

Voraussetzung für das Yolov5-Modell sind Koordinaten im Yolo-Format der Objekte, die für das Training relevant sind. Diese wurden im Schritt Data Preparation erstellt.




In [ ]:
### Github YOLOv5-Modell klonen

cd /Users/Julez/Downloads/detector
git clone https://github.com/ultralytics/yolov5.git
cd yolov5

# Installation Python-Abhängigkeiten
pip install -r requirements.txt


## 3.1 Data Preparation:
### 3.1.1 Umwandlung Yolo-Format
Zuerst wurden die Originalbilder und Annotationen aus dem Datensatz durch die folgenden 3 Skripte ins Yolo-Format konvertiert.

Die zugehörigen Dateien finden sich im Ordner: project-st24-nk_jr_js/counter_detection/data preparation

In [ ]:
### dataset_convert_annotations_test

import os
from PIL import Image

# Verzeichnis mit Bildern und Annotationsdateien
dataset_path = '/Users/julez/Downloads/detector/counter_detection/data_prep/testing'
output_path = '/Users/julez/Downloads/detector/counter_detection/yolov5_data/test/labels'

# Sicherstellen, dass das Ausgabe-Verzeichnis existiert
os.makedirs(output_path, exist_ok=True)

# Funktion zur Umwandlung ins YOLO-Format
def convert_to_yolo_format(meter_x, meter_y, meter_w, meter_h, image_width, image_height):
    x_center = (meter_x + meter_w / 2) / image_width
    y_center = (meter_y + meter_h / 2) / image_height
    width = meter_w / image_width
    height = meter_h / image_height
    return f"0 {x_center} {y_center} {width} {height}"

# Durchlaufen aller Annotationsdateien im Verzeichnis
for filename in os.listdir(dataset_path):
    if filename.endswith(".txt"):
        # Entsprechende Bilddatei finden
        image_filename = filename.replace('.txt', '.jpg')
        image_path = os.path.join(dataset_path, image_filename)

        if not os.path.exists(image_path):
            print(f"Bilddatei {image_filename} nicht gefunden.")
            continue

        # Bildgröße ermitteln
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        with open(os.path.join(dataset_path, filename), 'r') as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith('position:'):
                    parts = line.split()
                    meter_x = int(parts[1])
                    meter_y = int(parts[2])
                    meter_w = int(parts[3])
                    meter_h = int(parts[4])

                    yolo_annotation = convert_to_yolo_format(
                        meter_x, meter_y, meter_w, meter_h,
                        image_width, image_height
                    )

                    # Speicherung der YOLO-Format-Annotationsdatei
                    output_filename = os.path.join(output_path, filename)
                    with open(output_filename, 'w') as output_file:
                        output_file.write(yolo_annotation)

print("Umwandlung abgeschlossen.")


In [ ]:
### dataset_convert_annotations_train

import os
from PIL import Image

# Verzeichnis mit Bildern und Annotationsdateien
dataset_path = '/Users/julez/Downloads/detector/counter_detection/data_prep/training'
output_path = '/Users/julez/Downloads/detector/counter_detection/yolov5_data/train/labels'

# Sicherstellen, dass das Ausgabe-Verzeichnis existiert
os.makedirs(output_path, exist_ok=True)

# Funktion zur Umwandlung ins YOLO-Format
def convert_to_yolo_format(meter_x, meter_y, meter_w, meter_h, image_width, image_height):
    x_center = (meter_x + meter_w / 2) / image_width
    y_center = (meter_y + meter_h / 2) / image_height
    width = meter_w / image_width
    height = meter_h / image_height
    return f"0 {x_center} {y_center} {width} {height}"

# Durchlaufen aller Annotationsdateien im Verzeichnis
for filename in os.listdir(dataset_path):
    if filename.endswith(".txt"):
        # Entsprechende Bilddatei finden
        image_filename = filename.replace('.txt', '.jpg')
        image_path = os.path.join(dataset_path, image_filename)

        if not os.path.exists(image_path):
            print(f"Bilddatei {image_filename} nicht gefunden.")
            continue

        # Bildgröße ermitteln
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        with open(os.path.join(dataset_path, filename), 'r') as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith('position:'):
                    parts = line.split()
                    meter_x = int(parts[1])
                    meter_y = int(parts[2])
                    meter_w = int(parts[3])
                    meter_h = int(parts[4])

                    yolo_annotation = convert_to_yolo_format(
                        meter_x, meter_y, meter_w, meter_h,
                        image_width, image_height
                    )

                    # Speicherung der YOLO-Format-Annotationsdatei
                    output_filename = os.path.join(output_path, filename)
                    with open(output_filename, 'w') as output_file:
                        output_file.write(yolo_annotation)

print("Umwandlung abgeschlossen.")


In [ ]:
### dataset_convert_annotations_val

import os
from PIL import Image

# Verzeichnis mit Bildern und Annotationsdateien
dataset_path = '/Users/julez/Downloads/detector/counter_detection/data_prep/validation'
output_path = '/Users/julez/Downloads/detector/counter_detection/yolov5_data/val/labels'

# Sicherstellen, dass das Ausgabe-Verzeichnis existiert
os.makedirs(output_path, exist_ok=True)

# Funktion zur Umwandlung ins YOLO-Format
def convert_to_yolo_format(meter_x, meter_y, meter_w, meter_h, image_width, image_height):
    x_center = (meter_x + meter_w / 2) / image_width
    y_center = (meter_y + meter_h / 2) / image_height
    width = meter_w / image_width
    height = meter_h / image_height
    return f"0 {x_center} {y_center} {width} {height}"

# Durchlaufen aller Annotationsdateien im Verzeichnis
for filename in os.listdir(dataset_path):
    if filename.endswith(".txt"):
        # Entsprechende Bilddatei finden
        image_filename = filename.replace('.txt', '.jpg')
        image_path = os.path.join(dataset_path, image_filename)

        if not os.path.exists(image_path):
            print(f"Bilddatei {image_filename} nicht gefunden.")
            continue

        # Bildgröße ermitteln
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        with open(os.path.join(dataset_path, filename), 'r') as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith('position:'):
                    parts = line.split()
                    meter_x = int(parts[1])
                    meter_y = int(parts[2])
                    meter_w = int(parts[3])
                    meter_h = int(parts[4])

                    yolo_annotation = convert_to_yolo_format(
                        meter_x, meter_y, meter_w, meter_h,
                        image_width, image_height
                    )

                    # Speicherung der YOLO-Format-Annotationsdatei
                    output_filename = os.path.join(output_path, filename)
                    with open(output_filename, 'w') as output_file:
                        output_file.write(yolo_annotation)

print("Umwandlung abgeschlossen.")

In [ ]:
### Beispielhafte neu erstellte Struktur der YOLO-Annotationsdatei für das Bild 'meter0001.txt' :

0 0.4683760683760684 0.4480769230769231 0.18547008547008548 0.04278846153846154



### 3.1.2. Erstellung Training-Datensatz

Für den Trainingsdatensatz wurde der Ordner 'yolov5_data' erstellt. Die Bild- und Text-Dateien  wurden für unser weiteres Vorgehen hierzu händisch in Unterordner 'images' und 'labels' verschoben und in jeweils auf die Ordner 'train' (800 Bilder;  meter0001 - meter0800), 'test' (400 Bilder; meter0801 - meter1200) und 'val' (800 Bilder; meter1201 - meter2000) aufgeteilt.

In [ ]:
/Users/julez/downloads/detector/counter_detection/yolov5_data
├── train/
│   ├── images/
│   │   ├── meter0001.jpg
│   │   ├── meter0002jpg
│   │   └── ...
│   └── labels/
│       ├── meter0001.txt
│       ├── meter0002.txt
│       └── ...
├── val/
│   ├── images/
│   │   ├── meter1201.jpg
│   │   ├── meter1202.jpg
│   │   └── ...
│   └── labels/
│       ├── meter1201.txt
│       ├── meter1202.txt
│       └── ...
└── test/
    ├── images/
    │   ├── meter0801.jpg
    │   ├── meter0802.jpg
    │   └── ...
    └── labels/
        ├── meter0801.txt
        ├── meter0802.txt
        └── ...


## 3.2 Vorbereitung Trainingsprozess
### 3.2.1 Erstellen einer 'dataset_counter.yaml'-Datei als Grundlage für den Trainingsprozess

Ordner: project-st24-nk_jr_js/counter_detection/yaml files for yolov5



In [ ]:
### dataset_counter.yaml

train: /Users/julez/Downloads/detector/counter_detection/yolov5_data/train/images
val: /Users/julez/Downloads/detector/counter_detection/yolov5_data/val/images
test: /Users/julez/Downloads/detector/counter_detection/yolov5_data/test/labels

# Classes
nc: 1  # Anzahl der Klassen, passen Sie dies entsprechend Ihren Daten an
names: ['counter']  # Namen der Klassen, passen Sie dies entsprechend Ihren Daten an



## 3.3 Training YOLOv5-Modell

Das Modell wird durch den nachfolgendem Befehl mit dem Verweis auf die train.py Datei sowie der .yaml mit 50 Epochen gestartet. Dabei werden die Bilder auf 640x640 Pixel skaliert und das Training im Terminal gestartet. Die Ausgabe im Terminal während und nach dem Training sind im folgenden zu entnehmen. Der gespeicherte Trainings-Run befindet sich in der Datei:

project-st24-nk_jr_js/counter_detection/train and val runs yolov5/train/counter_detection_yolov5_Abgabe2

In [ ]:
### Startbefehl Training
python3 /Users/Julez/Downloads/detector/yolov5/train.py --img 640 --batch 16 --epochs 50 --data /Users/Julez/Downloads/detector/counter_detection/dataset_counter.yaml --weights yolov5s.pt --name counter_detection_yolov5_Abgabe2


In [ ]:
### Terminal Abschluss Training

train: weights=yolov5s.pt, cfg=, data=/Users/Julez/Downloads/detector/counter_detection/dataset_counter.yaml, hyp=../../../../Julez/Downloads/detector/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=../../../../Julez/Downloads/detector/yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../../../../Julez/Downloads/detector/yolov5/runs/train, name=counter_detection_yolov5_Abgabe, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 6 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
requirements: Ultralytics requirement ['thop>=0.1.1'] not found, attempting AutoUpdate...
Requirement already satisfied: thop>=0.1.1 in /opt/anaconda3/lib/python3.11/site-packages (0.1.1.post2209072238)
Requirement already satisfied: torch in /opt/anaconda3/lib/python3.11/site-packages (from thop>=0.1.1) (2.3.1)
Requirement already satisfied: filelock in /opt/anaconda3/lib/python3.11/site-packages (from torch->thop>=0.1.1) (3.13.1)
Requirement already satisfied: typing-extensions>=4.8.0 in /opt/anaconda3/lib/python3.11/site-packages (from torch->thop>=0.1.1) (4.9.0)
Requirement already satisfied: sympy in /opt/anaconda3/lib/python3.11/site-packages (from torch->thop>=0.1.1) (1.12)
Requirement already satisfied: networkx in /opt/anaconda3/lib/python3.11/site-packages (from torch->thop>=0.1.1) (3.1)
Requirement already satisfied: jinja2 in /opt/anaconda3/lib/python3.11/site-packages (from torch->thop>=0.1.1) (3.1.3)
Requirement already satisfied: fsspec in /opt/anaconda3/lib/python3.11/site-packages (from torch->thop>=0.1.1) (2023.10.0)
Requirement already satisfied: MarkupSafe>=2.0 in /opt/anaconda3/lib/python3.11/site-packages (from jinja2->torch->thop>=0.1.1) (2.1.3)
Requirement already satisfied: mpmath>=0.19 in /opt/anaconda3/lib/python3.11/site-packages (from sympy->torch->thop>=0.1.1) (1.3.0)

requirements: AutoUpdate success ✅ 1.8s, installed 1 package: ['thop>=0.1.1']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 v7.0-339-g150a1a31 Python-3.12.4 torch-2.3.1 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
Comet: run 'pip install comet_ml' to automatically track and visualize YOLOv5 🚀 runs in Comet
TensorBoard: Start with 'tensorboard --logdir ../../../../Julez/Downloads/detector/yolov5/runs/train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]
  2                -1  1     18816  models.common.C3                        [64, 64, 1]
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]
  4                -1  2    115712  models.common.C3                        [128, 128, 2]
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]
  6                -1  3    625152  models.common.C3                        [256, 256, 3]
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]
  8                -1  1   1182720  models.common.C3                        [512, 512, 1]
  9                -1  1    656896  models.common.SPPF                      [512, 512, 5]
 10                -1  1    131584  models.common.Conv                      [512, 256, 1, 1]
 11                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']
 12           [-1, 6]  1         0  models.common.Concat                    [1]
 13                -1  1    361984  models.common.C3                        [512, 256, 1, False]
 14                -1  1     33024  models.common.Conv                      [256, 128, 1, 1]
 15                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']
 16           [-1, 4]  1         0  models.common.Concat                    [1]
 17                -1  1     90880  models.common.C3                        [256, 128, 1, False]
 18                -1  1    147712  models.common.Conv                      [128, 128, 3, 2]
 19          [-1, 14]  1         0  models.common.Concat                    [1]
 20                -1  1    296448  models.common.C3                        [256, 256, 1, False]
 21                -1  1    590336  models.common.Conv                      [256, 256, 3, 2]
 22          [-1, 10]  1         0  models.common.Concat                    [1]
 23                -1  1   1182720  models.common.C3                        [512, 512, 1, False]
 24      [17, 20, 23]  1     16182  models.yolo.Detect                      [1, [[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]], [128, 256, 512]]
Model summary: 214 layers, 7022326 parameters, 7022326 gradients, 15.9 GFLOPs

Transferred 343/349 items from yolov5s.pt
optimizer: SGD(lr=0.01) with parameter groups 57 weight(decay=0.0), 60 weight(decay=0.0005), 60 bias
train: Scanning /Users/julez/Downloads/detector/counter_detection/yolov5_data/train/labels.cache... 800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]
val: Scanning /Users/julez/Downloads/detector/counter_detection/yolov5_data/val/labels.cache... 800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]

AutoAnchor: 3.59 anchors/target, 1.000 Best Possible Recall (BPR). Current anchors are a good fit to dataset ✅
Plotting labels to ../../../../Julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/labels.jpg...
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ../../../../Julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       0/49         0G     0.1034    0.02139          0         23        640: 100%|██████████| 50/50 [13:35<00:00, 16.32s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0%|          | 0/25 [00:00<?, ?it/s]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  16%|█▌        | 4/25 [00:49<04:16, 12.22s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  28%|██▊       | 7/25 [01:24<03:30, 11.68s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  44%|████▍     | 11/25 [02:11<02:46, 11.90s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  48%|████▊     | 12/25 [02:24<02:36, 12.02s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  52%|█████▏    | 13/25 [02:36<02:24, 12.08s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  56%|█████▌    | 14/25 [02:48<02:13, 12.15s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  64%|██████▍   | 16/25 [03:12<01:49, 12.12s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  68%|██████▊   | 17/25 [03:24<01:36, 12.10s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  72%|███████▏  | 18/25 [03:37<01:24, 12.11s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  76%|███████▌  | 19/25 [03:49<01:12, 12.10s/it]WARNING ⚠️ NMS time limit 2.100s exceeded
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [04:50<00:00, 11.62s/it]
                   all        800        800    0.00297      0.881      0.016    0.00611

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       1/49         0G    0.07203     0.0165          0         26        640: 100%|██████████| 50/50 [13:28<00:00, 16.17s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:48<00:00,  9.15s/it]
                   all        800        800    0.00393      0.986      0.233     0.0638

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       2/49         0G    0.05991    0.01553          0         24        640: 100%|██████████| 50/50 [10:48<00:00, 12.97s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:26<00:00,  8.25s/it]
                   all        800        800    0.00334          1      0.567      0.231

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       3/49         0G     0.0516    0.01489          0         24        640: 100%|██████████| 50/50 [10:49<00:00, 13.00s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:24<00:00,  8.19s/it]
                   all        800        800      0.696      0.622      0.719      0.259

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       4/49         0G     0.0454    0.01423          0         38        640: 100%|██████████| 50/50 [10:51<00:00, 13.02s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:23<00:00,  8.14s/it]
                   all        800        800      0.852      0.805      0.891      0.445

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       5/49         0G    0.04028    0.01223          0         34        640: 100%|██████████| 50/50 [10:49<00:00, 12.99s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:23<00:00,  8.14s/it]
                   all        800        800       0.91       0.93       0.96      0.408

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       6/49         0G    0.03823    0.01031          0         36        640: 100%|██████████| 50/50 [10:47<00:00, 12.94s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.11s/it]
                   all        800        800      0.953      0.966      0.982      0.467

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       7/49         0G    0.03576   0.008891          0         22        640: 100%|██████████| 50/50 [10:50<00:00, 13.01s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.09s/it]
                   all        800        800      0.963      0.954      0.973      0.422

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       8/49         0G    0.03415   0.008109          0         35        640: 100%|██████████| 50/50 [10:48<00:00, 12.97s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:23<00:00,  8.15s/it]
                   all        800        800      0.978       0.98      0.987      0.524

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
       9/49         0G    0.03323   0.007231          0         27        640: 100%|██████████| 50/50 [10:46<00:00, 12.92s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.09s/it]
                   all        800        800      0.985      0.987      0.991      0.633

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      10/49         0G    0.03181   0.006536          0         24        640: 100%|██████████| 50/50 [10:44<00:00, 12.88s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.10s/it]
                   all        800        800      0.978      0.986      0.991      0.635

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      11/49         0G    0.03194   0.006422          0         37        640: 100%|██████████| 50/50 [10:43<00:00, 12.88s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.12s/it]
                   all        800        800       0.98      0.994      0.993      0.571

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      12/49         0G    0.02959   0.005758          0         31        640: 100%|██████████| 50/50 [10:44<00:00, 12.90s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:23<00:00,  8.12s/it]
                   all        800        800      0.982      0.994      0.985      0.622

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      13/49         0G    0.02832   0.005441          0         23        640: 100%|██████████| 50/50 [10:44<00:00, 12.88s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.10s/it]
                   all        800        800       0.98      0.995      0.988      0.608

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      14/49         0G     0.0274   0.005447          0         25        640: 100%|██████████| 50/50 [10:45<00:00, 12.92s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:21<00:00,  8.08s/it]
                   all        800        800      0.978      0.989      0.989      0.641

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      15/49         0G    0.02809   0.005482          0         34        640: 100%|██████████| 50/50 [10:45<00:00, 12.90s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.10s/it]
                   all        800        800      0.983      0.994      0.992      0.697

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      16/49         0G    0.02679   0.005294          0         26        640: 100%|██████████| 50/50 [10:42<00:00, 12.86s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.11s/it]
                   all        800        800      0.984      0.995      0.993      0.657

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      17/49         0G     0.0265   0.004852          0         24        640: 100%|██████████| 50/50 [10:42<00:00, 12.84s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:21<00:00,  8.06s/it]
                   all        800        800      0.985      0.993      0.992      0.693

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      18/49         0G    0.02571   0.004875          0         28        640: 100%|██████████| 50/50 [10:44<00:00, 12.89s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:23<00:00,  8.13s/it]
                   all        800        800      0.984       0.99      0.987      0.687

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      19/49         0G    0.02558   0.004659          0         27        640: 100%|██████████| 50/50 [10:41<00:00, 12.83s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.11s/it]
                   all        800        800      0.985      0.994      0.993      0.718

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      20/49         0G    0.02465   0.004555          0         35        640: 100%|██████████| 50/50 [10:42<00:00, 12.85s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.03s/it]
                   all        800        800      0.983      0.997      0.993      0.614

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      21/49         0G    0.02468   0.004607          0         25        640: 100%|██████████| 50/50 [10:42<00:00, 12.85s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:21<00:00,  8.04s/it]
                   all        800        800      0.983      0.999      0.988      0.725

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      22/49         0G    0.02321   0.004387          0         30        640: 100%|██████████| 50/50 [10:37<00:00, 12.75s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:21<00:00,  8.06s/it]
                   all        800        800      0.985      0.997      0.985      0.741

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      23/49         0G    0.02371    0.00448          0         33        640: 100%|██████████| 50/50 [10:40<00:00, 12.81s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.02s/it]
                   all        800        800      0.985      0.998       0.99      0.741

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      24/49         0G    0.02242   0.004355          0         28        640: 100%|██████████| 50/50 [10:12<00:00, 12.25s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:14<00:00,  7.78s/it]
                   all        800        800      0.985      0.997      0.991      0.719

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      25/49         0G    0.02326   0.004028          0         19        640: 100%|██████████| 50/50 [10:35<00:00, 12.70s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.09s/it]
                   all        800        800      0.985      0.999       0.99       0.71

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      26/49         0G     0.0217    0.00424          0         25        640: 100%|██████████| 50/50 [10:42<00:00, 12.85s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.01s/it]
                   all        800        800      0.985      0.999      0.991      0.729

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      27/49         0G     0.0215   0.004065          0         30        640: 100%|██████████| 50/50 [10:37<00:00, 12.76s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.02s/it]
                   all        800        800      0.983      0.998      0.988      0.704

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      28/49         0G    0.02165   0.004031          0         37        640: 100%|██████████| 50/50 [10:35<00:00, 12.72s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.02s/it]
                   all        800        800      0.984      0.995      0.988      0.733

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      29/49         0G    0.02016   0.003902          0         32        640: 100%|██████████| 50/50 [10:38<00:00, 12.77s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.01s/it]
                   all        800        800      0.985      0.999       0.99      0.766

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      30/49         0G    0.01982   0.003791          0         29        640: 100%|██████████| 50/50 [10:40<00:00, 12.80s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:22<00:00,  8.08s/it]
                   all        800        800      0.985      0.997      0.992      0.756

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      31/49         0G    0.02049   0.003842          0         23        640: 100%|██████████| 50/50 [10:37<00:00, 12.75s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:21<00:00,  8.04s/it]
                   all        800        800      0.985      0.999      0.992      0.742

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      32/49         0G    0.01905   0.003678          0         30        640: 100%|██████████| 50/50 [10:36<00:00, 12.74s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.04s/it]
                   all        800        800      0.985      0.999      0.992       0.77

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      33/49         0G    0.01938   0.003709          0         24        640: 100%|██████████| 50/50 [10:40<00:00, 12.81s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.02s/it]
                   all        800        800      0.985      0.999       0.99      0.736

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      34/49         0G     0.0187   0.003571          0         23        640: 100%|██████████| 50/50 [10:39<00:00, 12.79s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.03s/it]
                   all        800        800      0.985          1      0.991      0.758

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      35/49         0G    0.01861   0.003729          0         31        640: 100%|██████████| 50/50 [10:40<00:00, 12.80s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.03s/it]
                   all        800        800      0.985          1       0.99       0.77

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      36/49         0G    0.01877   0.003677          0         28        640: 100%|██████████| 50/50 [10:39<00:00, 12.78s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:19<00:00,  7.97s/it]
                   all        800        800      0.985          1      0.991      0.724

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      37/49         0G    0.01764   0.003666          0         24        640: 100%|██████████| 50/50 [10:34<00:00, 12.70s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:18<00:00,  7.94s/it]
                   all        800        800      0.985          1      0.991      0.776

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      38/49         0G    0.01823   0.003603          0         33        640: 100%|██████████| 50/50 [10:30<00:00, 12.62s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:17<00:00,  7.91s/it]
                   all        800        800      0.985      0.996      0.991       0.77

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      39/49         0G     0.0179   0.003623          0         28        640: 100%|██████████| 50/50 [10:27<00:00, 12.56s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:17<00:00,  7.89s/it]
                   all        800        800      0.985      0.998      0.992      0.773

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      40/49         0G    0.01733   0.003655          0         25        640: 100%|██████████| 50/50 [10:27<00:00, 12.55s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:16<00:00,  7.87s/it]
                   all        800        800      0.985      0.998      0.993      0.777

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      41/49         0G      0.017   0.003626          0         32        640: 100%|██████████| 50/50 [10:30<00:00, 12.61s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:18<00:00,  7.96s/it]
                   all        800        800      0.985          1      0.993      0.778

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      42/49         0G     0.0167   0.003481          0         33        640: 100%|██████████| 50/50 [10:27<00:00, 12.54s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:17<00:00,  7.90s/it]
                   all        800        800      0.985      0.999      0.992      0.778

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      43/49         0G    0.01685   0.003505          0         31        640: 100%|██████████| 50/50 [10:27<00:00, 12.55s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:18<00:00,  7.95s/it]
                   all        800        800      0.985          1      0.991      0.781

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      44/49         0G    0.01663   0.003583          0         31        640: 100%|██████████| 50/50 [10:30<00:00, 12.61s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:19<00:00,  7.98s/it]
                   all        800        800      0.985          1      0.991      0.783

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      45/49         0G    0.01644   0.003617          0         32        640: 100%|██████████| 50/50 [10:30<00:00, 12.61s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:18<00:00,  7.94s/it]
                   all        800        800      0.985      0.999      0.991      0.781

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      46/49         0G    0.01543   0.003375          0         34        640: 100%|██████████| 50/50 [10:32<00:00, 12.65s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:19<00:00,  7.99s/it]
                   all        800        800      0.985      0.999      0.992      0.782

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      47/49         0G    0.01605    0.00342          0         20        640: 100%|██████████| 50/50 [10:34<00:00, 12.68s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:19<00:00,  7.98s/it]
                   all        800        800      0.985      0.999      0.992      0.786

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      48/49         0G    0.01565   0.003366          0         28        640: 100%|██████████| 50/50 [10:38<00:00, 12.78s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:20<00:00,  8.03s/it]
                   all        800        800      0.985          1      0.993      0.783

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
      49/49         0G     0.0157   0.003458          0         29        640: 100%|██████████| 50/50 [10:40<00:00, 12.81s/it]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:21<00:00,  8.04s/it]
                   all        800        800      0.985          1      0.993       0.79

50 epochs completed in 11.805 hours.
Optimizer stripped from ../../../../Julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/weights/last.pt, 14.3MB
Optimizer stripped from ../../../../Julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/weights/best.pt, 14.3MB

Validating ../../../../Julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/weights/best.pt...
Fusing layers...
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 25/25 [03:14<00:00,  7.79s/it]
                   all        800        800      0.985          1      0.993       0.79
Results saved to ../../../../Julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/weights/best.pt

## 3.4 Evaluation Training YOLOv5-Modell
Nach dem Trainingsprozess haben wir das Training evaluiert. Hierzu wurde die Validierungsdatei des Yolov5 Modells verwendet und die Ergebnisse im Ordner

project-st24-nk_jr_js/counter_detection/train and val runs yolov5/val hinterlegt.

Darüber hinaus wurde die Entwicklung des Modells im Traing geplottet und gespeichert, um Rückschlüsse auf den Erfolg des Traingsprozesses ziehen zu können (mAP & Box Loss). Diese Ergebnisse wurden im Ordner "plots" gespeichert, welcher sich in "counter_detection" befindet.

In [ ]:
python3 /Users/Julez/Downloads/detector/yolov5/val.py --weights /Users/Julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/weights/best.pt --data /Users/Julez/Downloads/detector/counter_detection/dataset_counter.yaml --img 640 --iou 0.6 --conf 0.001 --task test

Die nachfolgenden Skripte und Ergebnisse liegen im Ordner:

project-st24-nk_jr_js/counter_detection/plots for paper

In [ ]:
#Erstellen des mAP-Plots
import pandas as pd
import matplotlib.pyplot as plt

# CSV-Datei einlesen
csv_path = '/Users/julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/results.csv'
df = pd.read_csv(csv_path)

# Spaltennamen bereinigen
df.columns = df.columns.str.strip()

# Spaltennamen ausgeben, um zu überprüfen, ob 'epoch' existiert
print("Spaltennamen der bereinigten CSV-Datei:", df.columns)

# Diagramm erstellen, wenn 'epoch' existiert
if 'epoch' in df.columns:
    plt.figure(figsize=(10, 6))

    # Training und Validation MAE
    plt.plot(df['epoch'], df['metrics/mAP_0.5'], label='Train mAP_0.5', color='blue', linestyle='-')
    plt.plot(df['epoch'], df['metrics/mAP_0.5:0.95'], label='Validation mAP_0.5:0.95', color='red', linestyle='--')

    plt.title('Model Performance Yolov5 Counter Detection')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.legend()
    plt.grid(True)

    # Diagramm speichern und anzeigen
    plt.savefig('/Users/julez/Downloads/detector/counter_detection/plots/model_performance_mae.png')
    plt.show()
else:
    print("Die Spalte 'epoch' existiert nicht in der CSV-Datei.")


In [ ]:
#Plotten der Loss-Werte

import pandas as pd
import matplotlib.pyplot as plt

# CSV-Datei einlesen
csv_path = '/Users/julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/results.csv'
df = pd.read_csv(csv_path)

# Spaltennamen bereinigen
df.columns = df.columns.str.strip()

# Spaltennamen ausgeben, um zu überprüfen, ob 'epoch' existiert
print("Spaltennamen der bereinigten CSV-Datei:", df.columns)

# Diagramm erstellen, wenn 'epoch' existiert
if 'epoch' in df.columns:
    plt.figure(figsize=(10, 6))

    # Training und Validation Mean Absolute Error (MAE)
    plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', color='blue', linestyle='-')
    plt.plot(df['epoch'], df['val/box_loss'], label='Validation Box Loss', color='red', linestyle='--')

    plt.title('Model Performance Yolov5 Counter Detection')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Diagramm speichern und anzeigen
    plt.savefig('/Users/julez/Downloads/detector/counter_detection/plots/model_performance.png')
    plt.show()
else:
    print("Die Spalte 'epoch' existiert nicht in der CSV-Datei.")



## 3.5 Test YOLOv5-Modell starten inkl. Ausgabe der vorhergesagten Koordinaten

Um neue, bisher unbekannte Bilder zu testen kann der folgende Befehl ausgeführt werden. Jedoch muss der Ordnerpfad zu den zu testenden Dateien (--source) hierfür aktualisiert bzw. individuell angepasst werden. Gespeichert werden die vorhergesagten Counter sowie deren Annotationen im Ordner "test_predictions" unter dem Namen "results_test_final".

Pfad: project-st24-nk_jr_js/counter_detection/test_predictions

In [ ]:
python3 /Users/Julez/Downloads/detector/yolov5/detect.py --weights /Users/julez/Downloads/detector/yolov5/runs/train/counter_detection_yolov5_Abgabe2/weights/best.pt --source /Users/julez/Downloads/detector/counter_detection/yolov5_data/test/images --img 640 --conf 0.25 --save-txt --save-conf --project /Users/julez/Downloads/detector/counter_detection/test_predictions --name results_test_final

# 3.6 Nachverarbeitung der Detection Ergebnisse

Da in drei Fällen mehr als ein Counter erkannt wurde (was, wie sich am Ende herausgestellt hat auch richtig war), wurde ein Nachverarbeitungsskript erstellt, das sicherstellt, dass lediglich ein Counter pro Foto gefunden wird, da in der Praxis jedes erstellte Bild vorraussichtlich lediglich einen Counter haben wird. Das Ergebnis des Skripts wird im Ordner "filtered_labels" innerhalb des Ordners "results_test_final" gespeichert. Somit liegen nun bereinigte vorhergesagte Counter vor.

Pfad zum Skript: project-st24-nk_jr_js/counter_detection/python scripts/Post_test_Processing.py

Pfad zum Ergebnis: project-st24-nk_jr_js/counter_detection/test_predictions/results_test_final/labels/filtered_labels

In [ ]:
import os
import pandas as pd

# Pfade zu den Ausgabe- und Bildverzeichnissen
output_dir = '/Users/julez/Downloads/detector/counter_detection/test_predictions/results_test_final/labels'
filtered_output_dir = '/Users/julez/Downloads/detector/counter_detection/test_predictions/results_test_final/filtered_labels'

# Erstellen Sie das Ausgabe-Verzeichnis, wenn es nicht existiert
os.makedirs(filtered_output_dir, exist_ok=True)

# Alle Ausgabe-Textdateien durchlaufen
for label_file in os.listdir(output_dir):
    if label_file.endswith('.txt'):
        label_path = os.path.join(output_dir, label_file)

        # Einlesen der Textdatei
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # Wenn keine Zeilen vorhanden sind, überspringen
        if not lines:
            continue

        # Die Boxen und ihre Wahrscheinlichkeiten in einem DataFrame speichern
        boxes = []
        for line in lines:
            parts = line.strip().split()
            class_id, x_center, y_center, width, height, conf = parts
            boxes.append([class_id, float(x_center), float(y_center), float(width), float(height), float(conf)])

        df = pd.DataFrame(boxes, columns=['class_id', 'x_center', 'y_center', 'width', 'height', 'conf'])

        # Die Box mit der höchsten Wahrscheinlichkeit auswählen
        best_box = df.loc[df['conf'].idxmax()]

        # In einer neuen Datei speichern
        filtered_label_path = os.path.join(filtered_output_dir, label_file)
        with open(filtered_label_path, 'w') as f:
            f.write(f"{best_box['class_id']} {best_box['x_center']} {best_box['y_center']} {best_box['width']} {best_box['height']} {best_box['conf']}\n")

print(f'Filtered labels saved to: {filtered_output_dir}')


# 3.7 Evaluation der Testergebnisse durch IoU

Zur Evaluation der Ergebnisse, wurde ein Skript geschriben, welches die IoU für jede Vorhersage berechnet. Zudem wird die durchschnittliche, minimale und maximale IoU angegeben. Darüber hinaus zeigt das Ergebnis auch, wie viele Vorhersagen ein IoU unter 0.5 haben und somit nicht richtig erkannt wurden. Die drei betroffenen Bilder konnten anschließend händisch auf die Ursache untersucht werden.

Pfad zum Skript: project-st24-nk_jr_js/counter_detection/python scripts/Calculating_IoU.py

Pfad zum Ergebnis: counter_detection/test_predictions/results_test_final/iou_results.txt abgelegt.

In [ ]:
import os
import numpy as np
import cv2

def yolo_to_bbox(yolo_bbox, img_width, img_height):
    """Convert YOLO format to bounding box (x_min, y_min, x_max, y_max)."""
    x_center, y_center, width, height = yolo_bbox
    x_center, y_center, width, height = float(x_center), float(y_center), float(width), float(height)

    x_min = int((x_center - width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    x_max = int((x_center + width / 2) * img_width)
    y_max = int((y_center + height / 2) * img_height)

    return x_min, y_min, x_max, y_max

def compute_iou(box1, box2):
    """Compute IoU between two bounding boxes."""
    x_min1, y_min1, x_max1, y_max1 = box1
    x_min2, y_min2, x_max2, y_max2 = box2

    xi1 = max(x_min1, x_min2)
    yi1 = max(y_min1, y_min2)
    xi2 = min(x_max1, x_max2)
    yi2 = min(y_max1, y_max2)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x_max1 - x_min1) * (y_max1 - y_min1)
    box2_area = (x_max2 - x_min2) * (y_max2 - y_min2)

    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

# Pfade zu den vorhergesagten und den originalen Label-Dateien
predicted_labels_path = '/Users/julez/Downloads/detector/counter_detection/test_predictions/results_test_final/labels/filtered_labels'
original_labels_path = '/Users/julez/Downloads/detector/counter_detection/Test_images_model_combination/labels'
images_path = '/Users/julez/Downloads/detector/counter_detection/yolov5_data/test/labels'

iou_results = []
low_iou_files = []

# Vorhergesagte Label-Dateien durchlaufen
for label_file in os.listdir(predicted_labels_path):
    if label_file.endswith('.txt'):
        predicted_file_path = os.path.join(predicted_labels_path, label_file)
        original_file_path = os.path.join(original_labels_path, label_file)

        image_file = label_file.replace('.txt', '.jpg')  # Annahme: Bilddateien sind im JPG-Format
        img_path = os.path.join(images_path, image_file)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Warnung: Bild nicht gefunden oder kann nicht gelesen werden: {img_path}")
            continue

        img_height, img_width = img.shape[:2]

        with open(predicted_file_path, 'r') as f_pred, open(original_file_path, 'r') as f_orig:
            pred_lines = f_pred.readlines()
            orig_lines = f_orig.readlines()

            for pred_line, orig_line in zip(pred_lines, orig_lines):
                pred_parts = pred_line.strip().split()
                orig_parts = orig_line.strip().split()

                pred_bbox = yolo_to_bbox(pred_parts[1:5], img_width, img_height)
                orig_bbox = yolo_to_bbox(orig_parts[1:5], img_width, img_height)

                iou = compute_iou(pred_bbox, orig_bbox)
                iou_results.append((pred_bbox, orig_bbox, iou, label_file))

                if iou < 0.5:
                    low_iou_files.append(label_file)

# IoU Ergebnisse nach Meter-Nummer sortieren
iou_results.sort(key=lambda x: x[3])

# Durchschnittliche, minimale und maximale IoU berechnen
iou_values = [iou for _, _, iou, _ in iou_results]
average_iou = np.mean(iou_values)
min_iou = np.min(iou_values)
max_iou = np.max(iou_values)

# Ergebnisse in eine Textdatei schreiben
output_path = '/Users/julez/Downloads/detector/counter_detection/test_predictions/results_test_final/iou_results.txt'
with open(output_path, 'w') as f:
    for i, (pred_bbox, orig_bbox, iou, label_file) in enumerate(iou_results):
        f.write(f"{label_file}:\n")
        f.write(f"Vorhergesagte Koordinaten: {pred_bbox}\n")
        f.write(f"Tatsächliche Koordinaten: {orig_bbox}\n")
        f.write(f"IoU = {iou}\n\n")
    f.write(f"Durchschnittliche IoU: {average_iou}\n")
    f.write(f"Minimale IoU: {min_iou}\n")
    f.write(f"Maximale IoU: {max_iou}\n")
    f.write(f"Anzahl der Dateien mit IoU < 0,5: {len(low_iou_files)}\n")
    f.write("Dateien mit IoU < 0,5:\n")
    for file in low_iou_files:
        f.write(f"{file}\n")

print(f'Ergebnisse wurden in {output_path} gespeichert.')


**Ergebnis der IoU-Auswertung (Gesamtes Testing Set)**

Durchschnittliche IoU: 0.8635772997189792

Minimale IoU: 0.0

Maximale IoU: 0.9875325038623172

Anzahl der Dateien mit IoU < 0,5: 3

Dateien mit IoU < 0,5:

meter0847.txt
meter0809.txt
meter0870.txt



**Ergebnis der IoU-Auswertung (Kombination der Modelle)**

Durchschnittliche IoU: 0.8851765201650009

Minimale IoU: 0.5772017045454545

Maximale IoU: 0.9875325038623172

Anzahl der Dateien mit IoU < 0,5: 0

Dateien mit IoU < 0,5:

# 3.8 Bilder zuschneiden
Wie bereits oben erwähnt zielt unser Ansatz neben dem Trainingsprozess für die Counter Detection darauf ab, die Zählerregionen zu extrahieren und das Bild entsprechend dem Zähler zuzuschneiden. Nachdem das Training abgeschlossen und getestet wurde, haben wir über den folgenden Befehl die Bildregionen der Zähler extrahiert. Hierbei ist zu erwähnen, dass wir den Code so geschrieben haben, dass auf die größe der vorhergesagten Bounding Boxen ein bestimmter Aufschlag berechnet werrden kann. Wir haben uns für eine Vergrößerung der Bounding Boxes um 20% entschieden. Die zugeschnittenen Bilder werden im Ordner "test_output_cropped_img" in "counter_detection"  abgespeichert. Die so zugeschnittenen Bilder wurden dann an das CR-Net-Modell zur Digit Recognition übergeben.

Pfad zu den zugeschnittenen Bildern von test_final: project-st24-nk_jr_js/counter_detection/test_final_output_cropped_20p

Pfad zu den zugeschnittenen Bildern von test_combined (Kombination der Modelle):

*   Mit 20% Vergrößerung: project-st24-nk_jr_js/counter_detection/test_final_output_cropped_20p
*  Originalgröße: project-st24-nk_jr_js/counter_detection/test_combined_output_cropped_original




In [ ]:
import os
import cv2

def yolo_to_bbox(yolo_bbox, img_width, img_height, enlargement_factor=1.0):
    """Convert YOLO format to bounding box (x_min, y_min, x_max, y_max) and enlarge the box."""
    x_center, y_center, width, height = yolo_bbox
    x_center, y_center, width, height = float(x_center), float(y_center), float(width), float(height)

    width *= enlargement_factor
    height *= enlargement_factor

    x_min = int((x_center - width / 2) * img_width)
    y_min = int((y_center - height / 2) * img_height)
    x_max = int((x_center + width / 2) * img_width)
    y_max = int((y_center + height / 2) * img_height)

    # Sicherstellen, dass Koordinaten innerhalb der Bildgrenzen sind
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(img_width, x_max)
    y_max = min(img_height, y_max)

    return x_min, y_min, x_max, y_max

# Pfade zu den vorhergesagten Label-Dateien und zu den Originalbildern
predicted_labels_path = '/Users/julez/Downloads/detector/counter_detection/test_predictions/results_test_final/filtered_labels'
images_path = '/Users/julez/Downloads/detector/counter_detection/yolov5_data/test/images'
output_dir = '/Users/julez/Downloads/detector/counter_detection/test_output_cropped_img'
enlargement_factor = 1.2  # Vergrößerungsfaktor für die Bounding Box

# Sicherstellen, dass das Ausgabeverzeichnis existiert
os.makedirs(output_dir, exist_ok=True)

# Vorhergesagte Label-Dateien durchlaufen
for label_file in os.listdir(predicted_labels_path):
    if label_file.endswith('.txt'):
        with open(os.path.join(predicted_labels_path, label_file), 'r') as f:
            lines = f.readlines()

        # Holen des zugehörigen Bildes
        image_file = label_file.replace('.txt', '.jpg')  # Annahme: Bilddateien sind im JPG-Format
        img_path = os.path.join(images_path, image_file)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Warnung: Bild nicht gefunden oder kann nicht gelesen werden: {img_path}")
            continue

        img_height, img_width = img.shape[:2]

        # Bounding Box basierend auf den vorhergesagten Koordinaten zuschneiden
        for i, line in enumerate(lines):
            parts = line.strip().split()
            yolo_bbox = [float(p) for p in parts[1:5]]  # [class, x_center, y_center, width, height]
            x_min, y_min, x_max, y_max = yolo_to_bbox(yolo_bbox, img_width, img_height, enlargement_factor)

            # Ausschneiden des Bildes
            cropped_img = img[y_min:y_max, x_min:x_max]

            # Speichern des zugeschnittenen Bildes
            cropped_img_filename = f"{label_file.replace('.txt', '')}_cropped_{i+1}.jpg"
            cropped_img_path = os.path.join(output_dir, cropped_img_filename)
            cv2.imwrite(cropped_img_path, cropped_img)
            print(f'Gespeichert: {cropped_img_path}')


## 3.9 Vorbereitung Übergabe YOLOv5 -> CRNET
Um die Übergabe vom YOLOv5-Modell zum nächsten Projektschritt, dem CRNET-Modell, problemlos zu gestalten, wurde der Ordner 'digit_detection' erstellt, in dem mit den folgenden Befehlen die YOLO-Annotationsdateien der einzelnen 5 Digits für das CRNET-Modell erstellt wurden. Die zugeschnittenen Bilder, sowie die neuen .txt Dateien wurden in den Ordner '/Users/janriedel/Downloads/detector/digit_detection/yolov5_data' kopiert.

In [ ]:
### Testing Data Prep.py

import os
import cv2
import pandas as pd

# Funktion zum Ausschneiden der Bilder
def crop_image(image_path, bbox):
    img = cv2.imread(image_path)
    x, y, w, h = bbox
    cropped_img = img[y:y+h, x:x+w]
    return cropped_img

# Funktion zum Berechnen der neuen Koordinaten
def adjust_bbox(original_bbox, counter_bbox):
    x_min, y_min = counter_bbox[0], counter_bbox[1]
    new_bbox = [
        original_bbox[0] - x_min,
        original_bbox[1] - y_min,
        original_bbox[2],
        original_bbox[3]
    ]
    return new_bbox

# Funktion zum Konvertieren in YOLO-Format
def convert_to_yolo_format(bbox, img_width, img_height):
    x_min, y_min, width, height = bbox
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    width /= img_width
    height /= img_height
    return [x_center, y_center, width, height]

# Pfade
input_image_path = '/Users/julez/Downloads/detector/UFPR-AMR Dataset/testing'
annotation_file_path = '/Users/julez/Downloads/detector/UFPR-AMR Dataset/testing'
output_image_path = '/Users/julez/Downloads/detector/digit_detection/testing_data/images_crp'
new_annotations_path = '/Users/julez/Downloads/detector/digit_detection/testing_data/annotations_crp'
yolo_annotations_path = '/Users/julez/Downloads/detector/digit_detection/testing_data/annotations_yolo'

# Erstelle die Output-Ordner, falls nicht vorhanden
os.makedirs(output_image_path, exist_ok=True)
os.makedirs(new_annotations_path, exist_ok=True)
os.makedirs(yolo_annotations_path, exist_ok=True)

# Lese Annotationsdateien, schneide die Bilder und berechne neue Koordinaten
for annotation_file in os.listdir(annotation_file_path):
    if annotation_file.endswith('.txt'):
        with open(os.path.join(annotation_file_path, annotation_file), 'r') as f:
            lines = f.readlines()

        # Lese die Counter-Box-Koordinaten
        counter_line = lines[2].strip().split()
        try:
            counter_bbox = [int(counter_line[1]), int(counter_line[2]), int(counter_line[3]), int(counter_line[4])]
        except ValueError as e:
            print(f"Fehler beim Verarbeiten der Zeile: {counter_line}. Fehler: {e}")
            continue

        # Hole den Bildpfad
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(input_image_path, image_file)

        # Schneide das Bild aus
        cropped_img = crop_image(image_path, counter_bbox)

        # Speichere das ausgeschnittene Bild
        output_file = os.path.join(output_image_path, image_file)
        cv2.imwrite(output_file, cropped_img)

        new_lines = []

        # Berechne die neuen Koordinaten für jede Ziffer
        for i in range(3, 8):
            digit_line = lines[i].strip().split()
            try:
                original_bbox = [int(digit_line[j].replace(':', '')) for j in range(2, 6)]
            except ValueError as e:
                print(f"Fehler beim Verarbeiten der Zeile: {digit_line}. Fehler: {e}")
                continue
            new_bbox = adjust_bbox(original_bbox, counter_bbox)
            new_lines.append(f'{i-3} {new_bbox[0]} {new_bbox[1]} {new_bbox[2]} {new_bbox[3]}\n')

        # Speichere die neuen Annotationsdateien
        new_annotation_file = os.path.join(new_annotations_path, annotation_file)
        with open(new_annotation_file, 'w') as f:
            f.writelines(new_lines)

# Konvertiere die neuen Koordinaten in YOLO-Format
for annotation_file in os.listdir(new_annotations_path):
    if annotation_file.endswith('.txt'):
        with open(os.path.join(new_annotations_path, annotation_file), 'r') as f:
            lines = f.readlines()

        # Hole den Bildpfad
        image_file = annotation_file.replace('.txt', '.jpg')
        img_path = os.path.join(output_image_path, image_file)
        img = cv2.imread(img_path)

        # Bildgröße des zugeschnittenen Bildes
        img_height, img_width = img.shape[:2]

        yolo_lines = []

        for idx, line in enumerate(lines):
            parts = line.strip().split()
            try:
                bbox = [int(parts[i]) for i in range(1, 5)]
            except ValueError as e:
                print(f"Fehler beim Verarbeiten der Zeile: {parts}. Fehler: {e}")
                continue
            yolo_bbox = convert_to_yolo_format(bbox, img_width, img_height)
            yolo_lines.append(f'{parts[0]} {yolo_bbox[0]} {yolo_bbox[1]} {yolo_bbox[2]} {yolo_bbox[3]}\n')

        # Speichere die YOLO-Annotationsdateien
        yolo_annotation_file = os.path.join(yolo_annotations_path, annotation_file)
        with open(yolo_annotation_file, 'w') as f:
            f.writelines(yolo_lines)



In [ ]:
### Training Data Prep.py

import os
import cv2
import pandas as pd

# Funktion zum Ausschneiden der Bilder
def crop_image(image_path, bbox):
    img = cv2.imread(image_path)
    x, y, w, h = bbox
    cropped_img = img[y:y+h, x:x+w]
    return cropped_img

# Funktion zum Berechnen der neuen Koordinaten
def adjust_bbox(original_bbox, counter_bbox):
    x_min, y_min = counter_bbox[0], counter_bbox[1]
    new_bbox = [
        original_bbox[0] - x_min,
        original_bbox[1] - y_min,
        original_bbox[2],
        original_bbox[3]
    ]
    return new_bbox

# Funktion zum Konvertieren in YOLO-Format
def convert_to_yolo_format(bbox, img_width, img_height):
    x_min, y_min, width, height = bbox
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    width /= img_width
    height /= img_height
    return [x_center, y_center, width, height]

# Pfade
input_image_path = '/Users/julez/Downloads/detector/UFPR-AMR Dataset/training'
annotation_file_path = '/Users/julez/Downloads/detector/UFPR-AMR Dataset/training'
output_image_path = '/Users/julez/Downloads/detector/digit_detection/training_data/images_crp'
new_annotations_path = '/Users/julez/Downloads/detector/digit_detection/training_data/annotations_crp'
yolo_annotations_path = '/Users/julez/Downloads/detector/digit_detection/training_data/annotations_yolo'

# Erstelle die Output-Ordner, falls nicht vorhanden
os.makedirs(output_image_path, exist_ok=True)
os.makedirs(new_annotations_path, exist_ok=True)
os.makedirs(yolo_annotations_path, exist_ok=True)

# Lese Annotationsdateien, schneide die Bilder und berechne neue Koordinaten
for annotation_file in os.listdir(annotation_file_path):
    if annotation_file.endswith('.txt'):
        with open(os.path.join(annotation_file_path, annotation_file), 'r') as f:
            lines = f.readlines()

        # Lese die Counter-Box-Koordinaten
        counter_line = lines[2].strip().split()
        try:
            counter_bbox = [int(counter_line[1]), int(counter_line[2]), int(counter_line[3]), int(counter_line[4])]
        except ValueError as e:
            print(f"Fehler beim Verarbeiten der Zeile: {counter_line}. Fehler: {e}")
            continue

        # Hole den Bildpfad
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(input_image_path, image_file)

        # Schneide das Bild aus
        cropped_img = crop_image(image_path, counter_bbox)

        # Speichere das ausgeschnittene Bild
        output_file = os.path.join(output_image_path, image_file)
        cv2.imwrite(output_file, cropped_img)

        new_lines = []

        # Berechne die neuen Koordinaten für jede Ziffer
        for i in range(3, 8):
            digit_line = lines[i].strip().split()
            try:
                original_bbox = [int(digit_line[j].replace(':', '')) for j in range(2, 6)]
            except ValueError as e:
                print(f"Fehler beim Verarbeiten der Zeile: {digit_line}. Fehler: {e}")
                continue
            new_bbox = adjust_bbox(original_bbox, counter_bbox)
            new_lines.append(f'{i-3} {new_bbox[0]} {new_bbox[1]} {new_bbox[2]} {new_bbox[3]}\n')

        # Speichere die neuen Annotationsdateien
        new_annotation_file = os.path.join(new_annotations_path, annotation_file)
        with open(new_annotation_file, 'w') as f:
            f.writelines(new_lines)

# Konvertiere die neuen Koordinaten in YOLO-Format
for annotation_file in os.listdir(new_annotations_path):
    if annotation_file.endswith('.txt'):
        with open(os.path.join(new_annotations_path, annotation_file), 'r') as f:
            lines = f.readlines()

        # Hole den Bildpfad
        image_file = annotation_file.replace('.txt', '.jpg')
        img_path = os.path.join(output_image_path, image_file)
        img = cv2.imread(img_path)

        # Bildgröße des zugeschnittenen Bildes
        img_height, img_width = img.shape[:2]

        yolo_lines = []

        for idx, line in enumerate(lines):
            parts = line.strip().split()
            try:
                bbox = [int(parts[i]) for i in range(1, 5)]
            except ValueError as e:
                print(f"Fehler beim Verarbeiten der Zeile: {parts}. Fehler: {e}")
                continue
            yolo_bbox = convert_to_yolo_format(bbox, img_width, img_height)
            yolo_lines.append(f'{parts[0]} {yolo_bbox[0]} {yolo_bbox[1]} {yolo_bbox[2]} {yolo_bbox[3]}\n')

        # Speichere die YOLO-Annotationsdateien
        yolo_annotation_file = os.path.join(yolo_annotations_path, annotation_file)
        with open(yolo_annotation_file, 'w') as f:
            f.writelines(yolo_lines)


In [ ]:
### Validation Data Prep.py
import os
import cv2
import pandas as pd

# Funktion zum Ausschneiden der Bilder
def crop_image(image_path, bbox):
    img = cv2.imread(image_path)
    x, y, w, h = bbox
    cropped_img = img[y:y+h, x:x+w]
    return cropped_img

# Funktion zum Berechnen der neuen Koordinaten
def adjust_bbox(original_bbox, counter_bbox):
    x_min, y_min = counter_bbox[0], counter_bbox[1]
    new_bbox = [
        original_bbox[0] - x_min,
        original_bbox[1] - y_min,
        original_bbox[2],
        original_bbox[3]
    ]
    return new_bbox

# Funktion zum Konvertieren in YOLO-Format
def convert_to_yolo_format(bbox, img_width, img_height):
    x_min, y_min, width, height = bbox
    x_center = (x_min + width / 2) / img_width
    y_center = (y_min + height / 2) / img_height
    width /= img_width
    height /= img_height
    return [x_center, y_center, width, height]

# Pfade
input_image_path = '/Users/julez/Downloads/detector/UFPR-AMR Dataset/validation'
annotation_file_path = '/Users/julez/Downloads/detector/UFPR-AMR Dataset/validation'
output_image_path = '/Users/julez/Downloads/detector/digit_detection/validation_data/images_crp'
new_annotations_path = '/Users/julez/Downloads/detector/digit_detection/validation_data/annotations_crp'
yolo_annotations_path = '/Users/julez/Downloads/detector/digit_detection/validation_data/annotations_yolo'

# Erstelle die Output-Ordner, falls nicht vorhanden
os.makedirs(output_image_path, exist_ok=True)
os.makedirs(new_annotations_path, exist_ok=True)
os.makedirs(yolo_annotations_path, exist_ok=True)

# Lese Annotationsdateien, schneide die Bilder und berechne neue Koordinaten
for annotation_file in os.listdir(annotation_file_path):
    if annotation_file.endswith('.txt'):
        with open(os.path.join(annotation_file_path, annotation_file), 'r') as f:
            lines = f.readlines()

        # Lese die Counter-Box-Koordinaten
        counter_line = lines[2].strip().split()
        try:
            counter_bbox = [int(counter_line[1]), int(counter_line[2]), int(counter_line[3]), int(counter_line[4])]
        except ValueError as e:
            print(f"Fehler beim Verarbeiten der Zeile: {counter_line}. Fehler: {e}")
            continue

        # Hole den Bildpfad
        image_file = annotation_file.replace('.txt', '.jpg')
        image_path = os.path.join(input_image_path, image_file)

        # Schneide das Bild aus
        cropped_img = crop_image(image_path, counter_bbox)

        # Speichere das ausgeschnittene Bild
        output_file = os.path.join(output_image_path, image_file)
        cv2.imwrite(output_file, cropped_img)

        new_lines = []

        # Berechne die neuen Koordinaten für jede Ziffer
        for i in range(3, 8):
            digit_line = lines[i].strip().split()
            try:
                original_bbox = [int(digit_line[j].replace(':', '')) for j in range(2, 6)]
            except ValueError as e:
                print(f"Fehler beim Verarbeiten der Zeile: {digit_line}. Fehler: {e}")
                continue
            new_bbox = adjust_bbox(original_bbox, counter_bbox)
            new_lines.append(f'{i-3} {new_bbox[0]} {new_bbox[1]} {new_bbox[2]} {new_bbox[3]}\n')

        # Speichere die neuen Annotationsdateien
        new_annotation_file = os.path.join(new_annotations_path, annotation_file)
        with open(new_annotation_file, 'w') as f:
            f.writelines(new_lines)

# Konvertiere die neuen Koordinaten in YOLO-Format
for annotation_file in os.listdir(new_annotations_path):
    if annotation_file.endswith('.txt'):
        with open(os.path.join(new_annotations_path, annotation_file), 'r') as f:
            lines = f.readlines()

        # Hole den Bildpfad
        image_file = annotation_file.replace('.txt', '.jpg')
        img_path = os.path.join(output_image_path, image_file)
        img = cv2.imread(img_path)

        # Bildgröße des zugeschnittenen Bildes
        img_height, img_width = img.shape[:2]

        yolo_lines = []

        for idx, line in enumerate(lines):
            parts = line.strip().split()
            try:
                bbox = [int(parts[i]) for i in range(1, 5)]
            except ValueError as e:
                print(f"Fehler beim Verarbeiten der Zeile: {parts}. Fehler: {e}")
                continue
            yolo_bbox = convert_to_yolo_format(bbox, img_width, img_height)
            yolo_lines.append(f'{parts[0]} {yolo_bbox[0]} {yolo_bbox[1]} {yolo_bbox[2]} {yolo_bbox[3]}\n')

        # Speichere die YOLO-Annotationsdateien
        yolo_annotation_file = os.path.join(yolo_annotations_path, annotation_file)
        with open(yolo_annotation_file, 'w') as f:
            f.writelines(yolo_lines)



In [ ]:
### Beispielhafte neu erstellte Struktur der YOLO-Annotationen für die jeweiligen 5 Digits für  'meter0001.txt'

0 0.1175115207373272 0.48314606741573035 0.1935483870967742 0.8202247191011236
1 0.315668202764977 0.47752808988764045 0.17511520737327188 0.8089887640449438
2 0.4988479262672811 0.4747191011235955 0.1774193548387097 0.8033707865168539
3 0.6900921658986175 0.4747191011235955 0.1774193548387097 0.8033707865168539
4 0.8824884792626728 0.47752808988764045 0.15207373271889402 0.7640449438202247


# 4. CRNET Digit Detection

Im folgenden wird der Prozessschritt 'CRNET_digit_detection' beschrieben. Ziel war es, ein CRNET darauf zu trainieren, die jeweiligen Digits aus den gecroppten Bildern korrekt zu erkennen und vorherzusagen.

## 4.1. Data Preparation
### 4.1.1 Übernahme Datensatz aus 3.9 Digit Detection
Um den Traingsprozess zu starten, war ein erneutes Anpassen des Datensatzes notwendig. Hierfür wurde die erstellte Ordnerstruktur aus 3.9 ('/Users/janriedel/Downloads/detector/digit_detection/yolov5_data') in den folgenden Ordner kopiert: '/Users/janriedel/Downloads/detector/CRNET_digit_detection/data_preparation/image_crp'.

### 4.1.2 Erstellen neuer Textdateien
Das Trainingsmodell CRNET benötigt neben den bereits vorliegenden YOLO-Annotationsdaten der 5 Digits noch den IST-Wert des Zählerstandes auf dem Bild, um zuverlässig darauf trainiert zu werden, die richtigen Zählerstände zu predicten. Mit den folgenden Befehlen wurden zuerst die Reading-Scores aus den Originalen Textdateien des 'UFPR-AMR Dataset' extrahiert und zwischengespeichert, bevor die neuen Dateien mit den von Julian erstellten zusammengeführt wurden.


In [ ]:
#### extract_readings.py


import os

# Define the source and target directories
source_dir = '/Users/janriedel/Downloads/lbls/UFPR-AMR Dataset'
target_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/data_preparation/results/labels'

# Define the subdirectories
subdirs = ['training', 'testing', 'validation']

def process_file(file_path, target_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith('reading:'):
                reading_score = line.split(':')[1].strip()
                if len(reading_score) == 5:
                    with open(target_path, 'w') as output_file:
                        for digit in reading_score:
                            output_file.write(digit + '\n')
                break

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Process each file in the subdirectories
for subdir in subdirs:
    subdir_path = os.path.join(source_dir, subdir)
    for file_name in os.listdir(subdir_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(subdir_path, file_name)
            target_path = os.path.join(target_dir, file_name)
            process_file(file_path, target_path)

print("Processing complete!")



In [ ]:
### Beispielhafte neu erstellte Struktur der Reading-Scores für den jeweiligen Zählerstand für 'meter0001.txt'

0
0
0
0
1



In [ ]:
### labels_combined.py

import os

# Definiere die Verzeichnisse
source_readings_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/data_preparation/results/labels'
source_yolo_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/data_preparation/image_crp/yolov5_data'
target_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/data_preparation/newlabels'

# Definiere die Subdirectories
subdirs = ['train', 'test', 'val']

def read_reading_file(file_path):
    with open(file_path, 'r') as file:
        return file.readlines()

def read_yolo_file(file_path):
    with open(file_path, 'r') as file:
        return file.readlines()

def write_combined_file(reading_lines, yolo_lines, target_path):
    with open(target_path, 'w') as file:
        for line in reading_lines:
            file.write(line)
        for line in yolo_lines:
            file.write(line)

# Erstelle das Zielverzeichnis, wenn es nicht existiert
os.makedirs(target_dir, exist_ok=True)

# Prozessiere jede Datei in den Subdirectories
for subdir in subdirs:
    yolo_labels_path = os.path.join(source_yolo_dir, subdir, 'labels')
    for file_name in os.listdir(yolo_labels_path):
        if file_name.endswith('.txt'):
            reading_file_path = os.path.join(source_readings_dir, file_name)
            yolo_file_path = os.path.join(yolo_labels_path, file_name)
            target_file_path = os.path.join(target_dir, file_name)

            if os.path.exists(reading_file_path):
                reading_lines = read_reading_file(reading_file_path)
                yolo_lines = read_yolo_file(yolo_file_path)
                write_combined_file(reading_lines, yolo_lines, target_file_path)

print("Processing complete!")


In [ ]:
### Beispielhafte neu erstellte Struktur der Reading-Scores für den jeweiligen Zählerstand inkl. YOLO-Annotation für 'meter0001.txt'

0 0
1 0
2 0
3 0
4 1
0 0.1175115207373272 0.48314606741573035 0.1935483870967742 0.8202247191011236
1 0.315668202764977 0.47752808988764045 0.17511520737327188 0.8089887640449438
2 0.4988479262672811 0.4747191011235955 0.1774193548387097 0.8033707865168539
3 0.6900921658986175 0.4747191011235955 0.1774193548387097 0.8033707865168539
4 0.8824884792626728 0.47752808988764045 0.15207373271889402 0.7640449438202247



### 4.1.3 Erstellung CRNET Trainingsdatensatz 'yolov5_data'

Nachdem somit alle benötigten Dateien erstellt wurden, wurde der Datensatz händisch neu aufgeteilt. Entsprechend der gängigen Praxis für einen CRNET-Trainingsprozess haben wir uns für die folgende Aufteilung entschieden:

1. Training (75% der Bilder)
2. Testing (12,5%)
3. Validierung (12,5%)

also


1. Train: 1500 Bilder (meter0001 - meter0950 + meter1201 - meter1750)

2. Val: 250 Bilder (meter1751 - meter2000)

3. Test: 250 Bilder ( meter0951 - meter1200)

Die zugehörigen Textdateien wurden analog der folgenden Verzeichnisstruktur in den Ordner 'yolov5_data' kopiert.



In [ ]:
/Users/janriedel/Downloads/detector/CRNET_digit_detection/yolov5_data/
├── train/
│   ├── images/
│   │   ├── meter0001.jpg
│   │   ├── meter0002.jpg
│   │   └── ...
│   └── labels/
│       ├── meter0001.txt
│       ├── meter0002.txt
│       └── ...
├── val/
│   ├── images/
│   │   ├── meter1751.jpg
│   │   ├── meter1752.jpg
│   │   └── ...
│   └── labels/
│       ├── meter1751.txt
│       ├── meter1752.txt
│       └── ...
└── test/
    ├── images/
    │   ├── meter0951.jpg
    │   ├── meter0952.jpg
    │   └── ...
    └── labels/
        ├── meter0951.txt
        ├── meter0952.txt
        └── ...


## 4.2 Vorbereitung Trainingsprozess
Für den Trainingsprozess wurden 2 Dateien vorbereitet, anhand derer das Training durchgeführt werden kann: 'crnet.py' & 'train.crnet.py'


In [ ]:
### crnet.py

import torch
import torch.nn as nn

class CRNET(nn.Module):
    def __init__(self, num_classes=10, num_digits=5):
        super(CRNET, self).__init__()
        self.num_digits = num_digits
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifiers = nn.ModuleList([nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        ) for _ in range(num_digits)])

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        outputs = [classifier(x) for classifier in self.classifiers]
        return outputs

def crnet(num_classes=10, num_digits=5):
    model = CRNET(num_classes=num_classes, num_digits=num_digits)
    return model


In [ ]:
### train.crnet.py


import os
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

class CRNET(nn.Module):
    def __init__(self, num_classes=10, num_digits=5):
        super(CRNET, self).__init__()
        self.num_digits = num_digits
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifiers = nn.ModuleList([nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        ) for _ in range(num_digits)])

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        outputs = [classifier(x) for classifier in self.classifiers]
        return outputs

def crnet(num_classes=10, num_digits=5):
    model = CRNET(num_classes=num_classes, num_digits=num_digits)
    return model

class MeterDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        # Pfad zur Label-Datei
        label_path = os.path.join(self.label_dir, os.path.splitext(os.path.basename(img_path))[0] + '.txt')

        if not os.path.exists(label_path):
            raise FileNotFoundError(f"Label file not found: {label_path}")

        # Lese die Labels
        with open(label_path, 'r') as file:
            lines = file.readlines()
            digits = [int(line.split()[1]) for line in lines[:5]]  # Erste 5 Zeilen sind die Ziffern-Labels

        # Konvertieren zu einer Liste von Ziffern
        label = torch.tensor(digits, dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

def load_data(data_path, batch_size=32, img_size=224):
    print("Loading data...")
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = MeterDataset(os.path.join(data_path, 'train/images'), os.path.join(data_path, 'train/labels'), transform=transform)
    val_dataset = MeterDataset(os.path.join(data_path, 'val/images'), os.path.join(data_path, 'val/labels'), transform=transform)
    test_dataset = MeterDataset(os.path.join(data_path, 'test/images'), os.path.join(data_path, 'test/labels'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print("Data loaded.")
    return train_loader, val_loader, test_loader

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, device='cuda'):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = sum(criterion(output, labels[:, i]) for i, output in enumerate(outputs))
            _, preds = torch.max(torch.stack(outputs, dim=1), 2)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / (len(train_loader.dataset) * labels.size(1))

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = sum(criterion(output, labels[:, i]) for i, output in enumerate(outputs))
                _, preds = torch.max(torch.stack(outputs, dim=1), 2)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / (len(val_loader.dataset) * labels.size(1))

        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

    print(f'Best val Acc: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

def main():
    data_path = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/yolov5_data'
    num_classes = 10
    num_digits = 5
    batch_size = 32
    num_epochs = 200
    learning_rate = 0.0001  # Adjusted learning rate

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_loader, val_loader, test_loader = load_data(data_path, batch_size=batch_size, img_size=224)

    model = crnet(num_classes=num_classes, num_digits=num_digits)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, device=device)

    torch.save(model.state_dict(), 'crnet_dd.pth')

if __name__ == '__main__':
    main()


## 4.3 Training CRNET

Anhand der folgenden Parameter in train_crnet.py wurde das Modelltraining über 16 Stunden durchgeführt:  

    data_path = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/yolov5_data'

    num_classes = 10

    num_digits = 5

    batch_size = 32

    num_epochs = 200

    learning_rate = 0.0001

In [ ]:
### Ausführung train_crnet.py im Terminal


/usr/local/bin/python3 /Users/janriedel/Downloads/detector/CRNET_digit_detection/crnet.py
janriedel@MacBook-Pro-von-Jan-2 detector % /usr/local/bin/python3 /Users/janriedel/Downloads/detector/CRNET_digit_detection/crnet.p
y
janriedel@MacBook-Pro-von-Jan-2 detector % /usr/local/bin/python3 /Users/janriedel/Downloads/detector/CRNET_digit_detection/train.c
rnet.py
Loading data...
Data loaded.
Epoch 0/199
----------
Train Loss: 10.7458 Acc: 0.1773
Val Loss: 10.9406 Acc: 0.1528
Epoch 1/199
----------
Train Loss: 10.6240 Acc: 0.1843
Val Loss: 10.8332 Acc: 0.1544
Epoch 2/199
----------
Train Loss: 10.6068 Acc: 0.1907
Val Loss: 10.8460 Acc: 0.1448
Epoch 3/199
----------
Train Loss: 10.6055 Acc: 0.1853
Val Loss: 10.8282 Acc: 0.1536
Epoch 4/199
----------
Train Loss: 10.5754 Acc: 0.1892
Val Loss: 10.9268 Acc: 0.1488
Epoch 5/199
----------
Train Loss: 10.5541 Acc: 0.1959
Val Loss: 10.9138 Acc: 0.1544
Epoch 6/199
----------
Train Loss: 10.4971 Acc: 0.2028
Val Loss: 10.9022 Acc: 0.1568
Epoch 7/199
----------
Train Loss: 10.4685 Acc: 0.2108
Val Loss: 10.7977 Acc: 0.1824
Epoch 8/199
----------
Train Loss: 10.2027 Acc: 0.2384
Val Loss: 10.5033 Acc: 0.2032
Epoch 9/199
----------
Train Loss: 9.4858 Acc: 0.2976
Val Loss: 9.8820 Acc: 0.2616
Epoch 10/199
----------
Train Loss: 8.8061 Acc: 0.3521
Val Loss: 9.1169 Acc: 0.3272
Epoch 11/199
----------
Train Loss: 8.0959 Acc: 0.4120
Val Loss: 8.6259 Acc: 0.3832
Epoch 12/199
----------
Train Loss: 7.4224 Acc: 0.4660
Val Loss: 8.0940 Acc: 0.4240
Epoch 13/199
----------
Train Loss: 6.6098 Acc: 0.5295
Val Loss: 7.2257 Acc: 0.4816
Epoch 14/199
----------
Train Loss: 5.9469 Acc: 0.5839
Val Loss: 6.4322 Acc: 0.5624
Epoch 15/199
----------
Train Loss: 5.3427 Acc: 0.6253
Val Loss: 5.8911 Acc: 0.6040
Epoch 16/199
----------
Train Loss: 4.6693 Acc: 0.6812
Val Loss: 5.7007 Acc: 0.6160
Epoch 17/199
----------
Train Loss: 4.2100 Acc: 0.7128
Val Loss: 4.7486 Acc: 0.6736
Epoch 18/199
----------
Train Loss: 3.6133 Acc: 0.7523
Val Loss: 4.4879 Acc: 0.6920
Epoch 19/199
----------
Train Loss: 3.0733 Acc: 0.7921
Val Loss: 4.2740 Acc: 0.7184
Epoch 20/199
----------
Train Loss: 2.7909 Acc: 0.8136
Val Loss: 4.0623 Acc: 0.7344
Epoch 21/199
----------
Train Loss: 2.4001 Acc: 0.8388
Val Loss: 3.9335 Acc: 0.7528
Epoch 22/199
----------
Train Loss: 2.1371 Acc: 0.8599
Val Loss: 3.6985 Acc: 0.7584
Epoch 23/199
----------
Train Loss: 1.6999 Acc: 0.8873
Val Loss: 3.5565 Acc: 0.7704
Epoch 24/199
----------
Train Loss: 1.6144 Acc: 0.8921
Val Loss: 3.5986 Acc: 0.7832
Epoch 25/199
----------
Train Loss: 1.3646 Acc: 0.9125
Val Loss: 3.3614 Acc: 0.8016
Epoch 26/199
----------
Train Loss: 1.2160 Acc: 0.9188
Val Loss: 3.4549 Acc: 0.8144
Epoch 27/199
----------
Train Loss: 1.0439 Acc: 0.9329
Val Loss: 3.5099 Acc: 0.7960
Epoch 28/199
----------
Train Loss: 0.9885 Acc: 0.9356
Val Loss: 3.4834 Acc: 0.8032
Epoch 29/199
----------
Train Loss: 0.8180 Acc: 0.9460
Val Loss: 3.6607 Acc: 0.7984
Epoch 30/199
----------
Train Loss: 0.8031 Acc: 0.9465
Val Loss: 3.2917 Acc: 0.8096
Epoch 31/199
----------
Train Loss: 0.6922 Acc: 0.9564
Val Loss: 3.3820 Acc: 0.8248
Epoch 32/199
----------
Train Loss: 0.6239 Acc: 0.9603
Val Loss: 3.2555 Acc: 0.8352
Epoch 33/199
----------
Train Loss: 0.6350 Acc: 0.9581
Val Loss: 3.8473 Acc: 0.8096
Epoch 34/199
----------
Train Loss: 0.4916 Acc: 0.9671
Val Loss: 3.7236 Acc: 0.8272
Epoch 35/199
----------
Train Loss: 0.4553 Acc: 0.9680
Val Loss: 3.4018 Acc: 0.8368
Epoch 36/199
----------
Train Loss: 0.4518 Acc: 0.9708
Val Loss: 3.7118 Acc: 0.8280
Epoch 37/199
----------
Train Loss: 0.3910 Acc: 0.9753
Val Loss: 3.5979 Acc: 0.8360
Epoch 38/199
----------
Train Loss: 0.3686 Acc: 0.9763
Val Loss: 3.3338 Acc: 0.8360
Epoch 39/199
----------
Train Loss: 0.3641 Acc: 0.9761
Val Loss: 3.4003 Acc: 0.8368
Epoch 40/199
----------
Train Loss: 0.3242 Acc: 0.9805
Val Loss: 3.6923 Acc: 0.8240
Epoch 41/199
----------
Train Loss: 0.2745 Acc: 0.9821
Val Loss: 3.5391 Acc: 0.8472
Epoch 42/199
----------
Train Loss: 0.3329 Acc: 0.9769
Val Loss: 4.0197 Acc: 0.8320
Epoch 43/199
----------
Train Loss: 0.2815 Acc: 0.9815
Val Loss: 3.6645 Acc: 0.8528
Epoch 44/199
----------
Train Loss: 0.2964 Acc: 0.9819
Val Loss: 3.5667 Acc: 0.8456
Epoch 45/199
----------
Train Loss: 0.2985 Acc: 0.9808
Val Loss: 3.7410 Acc: 0.8408
Epoch 46/199
----------
Train Loss: 0.2892 Acc: 0.9819
Val Loss: 3.5132 Acc: 0.8272
Epoch 47/199
----------
Train Loss: 0.2268 Acc: 0.9857
Val Loss: 3.8584 Acc: 0.8392
Epoch 48/199
----------
Train Loss: 0.2229 Acc: 0.9859
Val Loss: 3.6166 Acc: 0.8384
Epoch 49/199
----------
Train Loss: 0.2662 Acc: 0.9829
Val Loss: 3.6957 Acc: 0.8416
Epoch 50/199
----------
Train Loss: 0.2235 Acc: 0.9863
Val Loss: 3.7006 Acc: 0.8488
Epoch 51/199
----------
Train Loss: 0.1975 Acc: 0.9876
Val Loss: 3.4276 Acc: 0.8536
Epoch 52/199
----------
Train Loss: 0.1975 Acc: 0.9879
Val Loss: 3.8105 Acc: 0.8520
Epoch 53/199
----------
Train Loss: 0.2006 Acc: 0.9867
Val Loss: 3.2930 Acc: 0.8632
Epoch 54/199
----------
Train Loss: 0.2063 Acc: 0.9860
Val Loss: 3.9365 Acc: 0.8368
Epoch 55/199
----------
Train Loss: 0.1705 Acc: 0.9895
Val Loss: 3.4254 Acc: 0.8624
Epoch 56/199
----------
Train Loss: 0.1853 Acc: 0.9872
Val Loss: 3.5504 Acc: 0.8552
Epoch 57/199
----------
Train Loss: 0.1701 Acc: 0.9887
Val Loss: 3.8119 Acc: 0.8592
Epoch 58/199
----------
Train Loss: 0.1721 Acc: 0.9883
Val Loss: 3.8438 Acc: 0.8560
Epoch 59/199
----------
Train Loss: 0.1563 Acc: 0.9901
Val Loss: 4.1307 Acc: 0.8440
Epoch 60/199
----------
Train Loss: 0.1439 Acc: 0.9915
Val Loss: 3.9639 Acc: 0.8416
Epoch 61/199
----------
Train Loss: 0.1708 Acc: 0.9877
Val Loss: 3.5430 Acc: 0.8632
Epoch 62/199
----------
Train Loss: 0.1600 Acc: 0.9896
Val Loss: 3.7694 Acc: 0.8560
Epoch 63/199
----------
Train Loss: 0.1590 Acc: 0.9897
Val Loss: 3.5022 Acc: 0.8536
Epoch 64/199
----------
Train Loss: 0.1842 Acc: 0.9891
Val Loss: 3.4556 Acc: 0.8424
Epoch 65/199
----------
Train Loss: 0.1663 Acc: 0.9891
Val Loss: 3.8181 Acc: 0.8408
Epoch 66/199
----------
Train Loss: 0.1573 Acc: 0.9908
Val Loss: 3.6829 Acc: 0.8536
Epoch 67/199
----------
Train Loss: 0.1383 Acc: 0.9913
Val Loss: 3.7802 Acc: 0.8576
Epoch 68/199
----------
Train Loss: 0.1092 Acc: 0.9927
Val Loss: 3.9376 Acc: 0.8504
Epoch 69/199
----------
Train Loss: 0.1140 Acc: 0.9941
Val Loss: 3.5327 Acc: 0.8600
Epoch 70/199
----------
Train Loss: 0.0894 Acc: 0.9941
Val Loss: 3.8637 Acc: 0.8600
Epoch 71/199
----------
Train Loss: 0.1313 Acc: 0.9917
Val Loss: 4.2060 Acc: 0.8480
Epoch 72/199
----------
Train Loss: 0.0973 Acc: 0.9941
Val Loss: 4.4529 Acc: 0.8352
Epoch 73/199
----------
Train Loss: 0.0831 Acc: 0.9952
Val Loss: 4.3934 Acc: 0.8424
Epoch 74/199
----------
Train Loss: 0.1028 Acc: 0.9944
Val Loss: 4.1003 Acc: 0.8472
Epoch 75/199
----------
Train Loss: 0.1209 Acc: 0.9927
Val Loss: 3.9172 Acc: 0.8472
Epoch 76/199
----------
Train Loss: 0.1063 Acc: 0.9932
Val Loss: 4.2988 Acc: 0.8464
Epoch 77/199
----------
Train Loss: 0.1102 Acc: 0.9936
Val Loss: 3.6447 Acc: 0.8680
Epoch 78/199
----------
Train Loss: 0.0825 Acc: 0.9951
Val Loss: 4.5085 Acc: 0.8472
Epoch 79/199
----------
Train Loss: 0.1338 Acc: 0.9915
Val Loss: 3.6724 Acc: 0.8536
Epoch 80/199
----------
Train Loss: 0.1146 Acc: 0.9929
Val Loss: 4.7838 Acc: 0.8512
Epoch 81/199
----------
Train Loss: 0.1181 Acc: 0.9925
Val Loss: 4.0236 Acc: 0.8584
Epoch 82/199
----------
Train Loss: 0.1120 Acc: 0.9935
Val Loss: 3.6883 Acc: 0.8560
Epoch 83/199
----------
Train Loss: 0.0894 Acc: 0.9941
Val Loss: 4.0497 Acc: 0.8584
Epoch 84/199
----------
Train Loss: 0.0833 Acc: 0.9947
Val Loss: 4.4444 Acc: 0.8656
Epoch 85/199
----------
Train Loss: 0.1172 Acc: 0.9925
Val Loss: 4.0381 Acc: 0.8264
Epoch 86/199
----------
Train Loss: 0.1344 Acc: 0.9916
Val Loss: 4.3261 Acc: 0.8448
Epoch 87/199
----------
Train Loss: 0.0942 Acc: 0.9952
Val Loss: 4.1724 Acc: 0.8496
Epoch 88/199
----------
Train Loss: 0.0874 Acc: 0.9951
Val Loss: 4.3929 Acc: 0.8464
Epoch 89/199
----------
Train Loss: 0.1036 Acc: 0.9940
Val Loss: 4.2739 Acc: 0.8536
Epoch 90/199
----------
Train Loss: 0.0898 Acc: 0.9947
Val Loss: 3.6251 Acc: 0.8616
Epoch 91/199
----------
Train Loss: 0.1470 Acc: 0.9909
Val Loss: 4.1515 Acc: 0.8424
Epoch 92/199
----------
Train Loss: 0.1257 Acc: 0.9931
Val Loss: 3.8642 Acc: 0.8568
Epoch 93/199
----------
Train Loss: 0.0706 Acc: 0.9960
Val Loss: 3.5689 Acc: 0.8736
Epoch 94/199
----------
Train Loss: 0.0643 Acc: 0.9960
Val Loss: 4.4681 Acc: 0.8456
Epoch 95/199
----------
Train Loss: 0.0560 Acc: 0.9963
Val Loss: 3.9015 Acc: 0.8600
Epoch 96/199
----------
Train Loss: 0.0714 Acc: 0.9956
Val Loss: 3.9695 Acc: 0.8520
Epoch 97/199
----------
Train Loss: 0.0697 Acc: 0.9959
Val Loss: 4.3993 Acc: 0.8568
Epoch 98/199
----------
Train Loss: 0.0525 Acc: 0.9964
Val Loss: 4.3262 Acc: 0.8480
Epoch 99/199
----------
Train Loss: 0.0715 Acc: 0.9948
Val Loss: 4.5708 Acc: 0.8472
Epoch 100/199
----------
Train Loss: 0.0531 Acc: 0.9969
Val Loss: 3.6358 Acc: 0.8688
Epoch 101/199
----------
Train Loss: 0.0405 Acc: 0.9981
Val Loss: 4.3622 Acc: 0.8728
Epoch 102/199
----------
Train Loss: 0.0809 Acc: 0.9951
Val Loss: 3.6977 Acc: 0.8648
Epoch 103/199
----------
Train Loss: 0.0428 Acc: 0.9972
Val Loss: 3.9639 Acc: 0.8592
Epoch 104/199
----------
Train Loss: 0.0685 Acc: 0.9964
Val Loss: 3.6257 Acc: 0.8712
Epoch 105/199
----------
Train Loss: 0.0716 Acc: 0.9956
Val Loss: 5.1643 Acc: 0.8480
Epoch 106/199
----------
Train Loss: 0.1130 Acc: 0.9932
Val Loss: 4.3079 Acc: 0.8464
Epoch 107/199
----------
Train Loss: 0.1521 Acc: 0.9909
Val Loss: 3.7304 Acc: 0.8640
Epoch 108/199
----------
Train Loss: 0.0744 Acc: 0.9948
Val Loss: 4.0700 Acc: 0.8656
Epoch 109/199
----------
Train Loss: 0.0454 Acc: 0.9972
Val Loss: 3.7969 Acc: 0.8712
Epoch 110/199
----------
Train Loss: 0.0716 Acc: 0.9964
Val Loss: 4.0396 Acc: 0.8560
Epoch 111/199
----------
Train Loss: 0.0470 Acc: 0.9973
Val Loss: 4.4005 Acc: 0.8512
Epoch 112/199
----------
Train Loss: 0.0589 Acc: 0.9961
Val Loss: 3.8871 Acc: 0.8592
Epoch 113/199
----------
Train Loss: 0.0578 Acc: 0.9955
Val Loss: 4.2340 Acc: 0.8560
Epoch 114/199
----------
Train Loss: 0.0683 Acc: 0.9956
Val Loss: 3.6663 Acc: 0.8536
Epoch 115/199
----------
Train Loss: 0.0731 Acc: 0.9953
Val Loss: 5.2244 Acc: 0.8392
Epoch 116/199
----------
Train Loss: 0.1139 Acc: 0.9936
Val Loss: 3.6931 Acc: 0.8640
Epoch 117/199
----------
Train Loss: 0.1225 Acc: 0.9924
Val Loss: 3.5568 Acc: 0.8704
Epoch 118/199
----------
Train Loss: 0.0655 Acc: 0.9952
Val Loss: 3.7940 Acc: 0.8544
Epoch 119/199
----------
Train Loss: 0.0519 Acc: 0.9969
Val Loss: 3.8384 Acc: 0.8664
Epoch 120/199
----------
Train Loss: 0.0893 Acc: 0.9951
Val Loss: 4.3974 Acc: 0.8536
Epoch 121/199
----------
Train Loss: 0.0714 Acc: 0.9945
Val Loss: 3.2843 Acc: 0.8696
Epoch 122/199
----------
Train Loss: 0.0366 Acc: 0.9979
Val Loss: 3.9227 Acc: 0.8696
Epoch 123/199
----------
Train Loss: 0.0445 Acc: 0.9971
Val Loss: 3.8344 Acc: 0.8640
Epoch 124/199
----------
Train Loss: 0.0412 Acc: 0.9975
Val Loss: 3.5687 Acc: 0.8640
Epoch 125/199
----------
Train Loss: 0.0764 Acc: 0.9951
Val Loss: 4.1810 Acc: 0.8736
Epoch 126/199
----------
Train Loss: 0.0628 Acc: 0.9961
Val Loss: 4.9766 Acc: 0.8528
Epoch 127/199
----------
Train Loss: 0.0727 Acc: 0.9955
Val Loss: 3.9725 Acc: 0.8616
Epoch 128/199
----------
Train Loss: 0.0548 Acc: 0.9968
Val Loss: 3.7859 Acc: 0.8680
Epoch 129/199
----------
Train Loss: 0.0718 Acc: 0.9951
Val Loss: 3.7838 Acc: 0.8720
Epoch 130/199
----------
Train Loss: 0.0709 Acc: 0.9956
Val Loss: 3.8620 Acc: 0.8744
Epoch 131/199
----------
Train Loss: 0.0457 Acc: 0.9977
Val Loss: 4.1133 Acc: 0.8568
Epoch 132/199
----------
Train Loss: 0.0467 Acc: 0.9977
Val Loss: 4.1614 Acc: 0.8648
Epoch 133/199
----------
Train Loss: 0.0276 Acc: 0.9983
Val Loss: 3.8793 Acc: 0.8696
Epoch 134/199
----------
Train Loss: 0.0450 Acc: 0.9975
Val Loss: 4.2290 Acc: 0.8640
Epoch 135/199
----------
Train Loss: 0.0498 Acc: 0.9971
Val Loss: 3.7941 Acc: 0.8656
Epoch 136/199
----------
Train Loss: 0.0427 Acc: 0.9968
Val Loss: 3.7803 Acc: 0.8824
Epoch 137/199
----------
Train Loss: 0.0625 Acc: 0.9965
Val Loss: 4.4408 Acc: 0.8664
Epoch 138/199
----------
Train Loss: 0.0827 Acc: 0.9943
Val Loss: 3.7804 Acc: 0.8672
Epoch 139/199
----------
Train Loss: 0.0618 Acc: 0.9959
Val Loss: 4.7025 Acc: 0.8568
Epoch 140/199
----------
Train Loss: 0.0630 Acc: 0.9968
Val Loss: 4.1623 Acc: 0.8720
Epoch 141/199
----------
Train Loss: 0.0704 Acc: 0.9959
Val Loss: 3.8103 Acc: 0.8600
Epoch 142/199
----------
Train Loss: 0.0568 Acc: 0.9963
Val Loss: 3.8282 Acc: 0.8720
Epoch 143/199
----------
Train Loss: 0.0508 Acc: 0.9971
Val Loss: 3.2581 Acc: 0.8712
Epoch 144/199
----------
Train Loss: 0.0500 Acc: 0.9976
Val Loss: 4.0524 Acc: 0.8840
Epoch 145/199
----------
Train Loss: 0.0601 Acc: 0.9961
Val Loss: 3.7267 Acc: 0.8704
Epoch 146/199
----------
Train Loss: 0.0608 Acc: 0.9963
Val Loss: 3.6881 Acc: 0.8640
Epoch 147/199
----------
Train Loss: 0.0771 Acc: 0.9961
Val Loss: 4.5519 Acc: 0.8600
Epoch 148/199
----------
Train Loss: 0.0414 Acc: 0.9971
Val Loss: 4.1598 Acc: 0.8688
Epoch 149/199
----------
Train Loss: 0.0359 Acc: 0.9977
Val Loss: 4.1538 Acc: 0.8776
Epoch 150/199
----------
Train Loss: 0.0677 Acc: 0.9960
Val Loss: 4.0481 Acc: 0.8608
Epoch 151/199
----------
Train Loss: 0.0648 Acc: 0.9961
Val Loss: 4.2970 Acc: 0.8648
Epoch 152/199
----------
Train Loss: 0.0569 Acc: 0.9967
Val Loss: 4.2493 Acc: 0.8712
Epoch 153/199
----------
Train Loss: 0.0506 Acc: 0.9964
Val Loss: 3.9948 Acc: 0.8744
Epoch 154/199
----------
Train Loss: 0.0686 Acc: 0.9952
Val Loss: 4.4103 Acc: 0.8600
Epoch 155/199
----------
Train Loss: 0.0673 Acc: 0.9963
Val Loss: 3.9269 Acc: 0.8664
Epoch 156/199
----------
Train Loss: 0.0473 Acc: 0.9965
Val Loss: 3.8031 Acc: 0.8736
Epoch 157/199
----------
Train Loss: 0.0511 Acc: 0.9975
Val Loss: 3.9042 Acc: 0.8656
Epoch 158/199
----------
Train Loss: 0.0378 Acc: 0.9980
Val Loss: 3.8877 Acc: 0.8768
Epoch 159/199
----------
Train Loss: 0.0764 Acc: 0.9967
Val Loss: 3.7665 Acc: 0.8728
Epoch 160/199
----------
Train Loss: 0.0606 Acc: 0.9967
Val Loss: 3.9160 Acc: 0.8688
Epoch 161/199
----------
Train Loss: 0.0464 Acc: 0.9973
Val Loss: 4.1750 Acc: 0.8768
Epoch 162/199
----------
Train Loss: 0.0500 Acc: 0.9972
Val Loss: 3.9411 Acc: 0.8728
Epoch 163/199
----------
Train Loss: 0.0344 Acc: 0.9984
Val Loss: 3.9328 Acc: 0.8552
Epoch 164/199
----------
Train Loss: 0.0355 Acc: 0.9972
Val Loss: 3.8958 Acc: 0.8736
Epoch 165/199
----------
Train Loss: 0.0334 Acc: 0.9980
Val Loss: 3.7026 Acc: 0.8696
Epoch 166/199
----------
Train Loss: 0.0374 Acc: 0.9973
Val Loss: 4.3964 Acc: 0.8656
Epoch 167/199
----------
Train Loss: 0.0649 Acc: 0.9967
Val Loss: 4.2886 Acc: 0.8656
Epoch 168/199
----------
Train Loss: 0.0398 Acc: 0.9971
Val Loss: 4.5983 Acc: 0.8568
Epoch 169/199
----------
Train Loss: 0.0386 Acc: 0.9977
Val Loss: 4.2096 Acc: 0.8616
Epoch 170/199
----------
Train Loss: 0.0536 Acc: 0.9967
Val Loss: 4.2267 Acc: 0.8664
Epoch 171/199
----------
Train Loss: 0.0766 Acc: 0.9955
Val Loss: 4.1794 Acc: 0.8568
Epoch 172/199
----------
Train Loss: 0.0549 Acc: 0.9976
Val Loss: 3.5780 Acc: 0.8640
Epoch 173/199
----------
Train Loss: 0.0482 Acc: 0.9975
Val Loss: 3.9927 Acc: 0.8640
Epoch 174/199
----------
Train Loss: 0.0469 Acc: 0.9969
Val Loss: 4.3735 Acc: 0.8608
Epoch 175/199
----------
Train Loss: 0.0585 Acc: 0.9971
Val Loss: 3.7796 Acc: 0.8800
Epoch 176/199
----------
Train Loss: 0.0360 Acc: 0.9976
Val Loss: 3.8465 Acc: 0.8672
Epoch 177/199
----------
Train Loss: 0.0754 Acc: 0.9953
Val Loss: 3.7255 Acc: 0.8704
Epoch 178/199
----------
Train Loss: 0.0521 Acc: 0.9973
Val Loss: 3.4099 Acc: 0.8688
Epoch 179/199
----------
Train Loss: 0.0438 Acc: 0.9972
Val Loss: 3.9390 Acc: 0.8680
Epoch 180/199
----------
Train Loss: 0.0390 Acc: 0.9969
Val Loss: 3.8571 Acc: 0.8784
Epoch 181/199
----------
Train Loss: 0.0653 Acc: 0.9967
Val Loss: 3.6765 Acc: 0.8656
Epoch 182/199
----------
Train Loss: 0.0663 Acc: 0.9965
Val Loss: 4.1639 Acc: 0.8536
Epoch 183/199
----------
Train Loss: 0.0617 Acc: 0.9968
Val Loss: 3.6442 Acc: 0.8744
Epoch 184/199
----------
Train Loss: 0.0390 Acc: 0.9975
Val Loss: 3.6672 Acc: 0.8760
Epoch 185/199
----------
Train Loss: 0.0470 Acc: 0.9971
Val Loss: 4.3677 Acc: 0.8672
Epoch 186/199
----------
Train Loss: 0.0451 Acc: 0.9973
Val Loss: 4.0510 Acc: 0.8648
Epoch 187/199
----------
Train Loss: 0.0632 Acc: 0.9961
Val Loss: 3.4779 Acc: 0.8864
Epoch 188/199
----------
Train Loss: 0.0433 Acc: 0.9973
Val Loss: 4.0078 Acc: 0.8672
Epoch 189/199
----------
Train Loss: 0.0566 Acc: 0.9976
Val Loss: 3.6331 Acc: 0.8712
Epoch 190/199
----------
Train Loss: 0.0438 Acc: 0.9973
Val Loss: 3.6330 Acc: 0.8832
Epoch 191/199
----------
Train Loss: 0.0245 Acc: 0.9980
Val Loss: 4.1123 Acc: 0.8544
Epoch 192/199
----------
Train Loss: 0.0239 Acc: 0.9985
Val Loss: 3.6755 Acc: 0.8784
Epoch 193/199
----------
Train Loss: 0.0356 Acc: 0.9984
Val Loss: 3.7750 Acc: 0.8760
Epoch 194/199
----------
Train Loss: 0.0283 Acc: 0.9976
Val Loss: 4.2314 Acc: 0.8672
Epoch 195/199
----------
Train Loss: 0.0367 Acc: 0.9976
Val Loss: 3.9438 Acc: 0.8768
Epoch 196/199
----------
Train Loss: 0.0232 Acc: 0.9987
Val Loss: 4.0174 Acc: 0.8648
Epoch 197/199
----------
Train Loss: 0.0547 Acc: 0.9980
Val Loss: 3.5034 Acc: 0.8744
Epoch 198/199
----------
Train Loss: 0.0161 Acc: 0.9989
Val Loss: 4.1435 Acc: 0.8680
Epoch 199/199
----------
Train Loss: 0.0219 Acc: 0.9987
Val Loss: 3.9682 Acc: 0.8800
Best val Acc: 0.8864

Um die hier dargestellt Accuracy des Modells zu visualisieren, wurde das folgende Skript erstellt. Dieses basierte auf einer txt-Datei, die händisch anhand des o.g. Terminal-Outputs erstellt wurde.

In [ ]:
import matplotlib.pyplot as plt

# Pfad zur Datei
file_path = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/crnet_train_result.txt'

# Listen für Epochen, Trainingsgenauigkeit und Validierungsgenauigkeit
epochs = []
train_acc = []
val_acc = []

# Datei einlesen und Werte extrahieren
with open(file_path, 'r') as file:
    for line in file:
        if 'Epoch' in line:
            epoch = int(line.strip().split()[1].split('/')[0])
            if epoch > 0:  # Epoche 0 überspringen
                epochs.append(epoch)
        elif 'Train Loss' in line:
            train_accuracy = float(line.strip().split('Acc: ')[-1])
            train_acc.append(train_accuracy)
        elif 'Val Loss' in line:
            val_accuracy = float(line.strip().split('Acc: ')[-1])
            val_acc.append(val_accuracy)

# Sicherstellen, dass die Länge der Listen 199 beträgt
assert len(epochs) == 199, "Epochs list length is not 199."
assert len(train_acc) == 199, "Train accuracy list length is not 199."
assert len(val_acc) == 199, "Validation accuracy list length is not 199."

# Plot erstellen
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_acc, 'b', label='Train Accuracy')
plt.plot(epochs, val_acc, 'r--', label='Validation Accuracy')
plt.title('Model Accuracy CR-Net')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0, 1])
plt.grid(True)

# Diagramm speichern
plt.savefig('/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/model_accuracy_CRNet.png')

# Diagramm anzeigen
plt.show()


In [ ]:
### Hochladen der Modelldatei drnet_dd.pth
###Um die Datei 'crnet_dd.pth' ('/Users/janriedel/Downloads/detector/crnet_dd.pth') mit 2,4 GB zur Abgabe zur Verfügung stellen zu können, wurde die Erweiterung 'Git Large Files' (https://git-lfs.com/ -> '/Users/janriedel/Downloads/detector/git-lfs-3.5.1') installiert und ausgeführt.


cd /Users/janriedel/Documents/GitHub/project-st24-nk_jr_js

git lfs track "*.pth"

git add .gitattributes
git commit -m "Track .pth files with LFS"

cp /Users/janriedel/Downloads/detector/crnet_dd.pth /Users/janriedel/Documents/GitHub/project-st24-nk_jr_js

git add crnet_dd.pth
git commit -m "Add crnet_dd.pth file"
git push origin main





## 4.4. Predictions Neue Bilder / Evaluation Trainingsmodell

Das trainierte Modell wurde durch das folgende Skript getestet. Testgrundlage waren die Testbilder im Ordner 'yolov5_data'.

In [ ]:
### Predictions Bilder Test und speichern der Predictions 'prediction_test.py'

import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Definiere die CRNET-Klasse und die crnet-Funktion
class CRNET(nn.Module):
    def __init__(self, num_classes=10, num_digits=5):
        super(CRNET, self).__init__()
        self.num_digits = num_digits
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifiers = nn.ModuleList([nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        ) for _ in range(num_digits)])

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        outputs = [classifier(x) for classifier in self.classifiers]
        return outputs

def crnet(num_classes=10, num_digits=5):
    model = CRNET(num_classes=num_classes, num_digits=num_digits)
    return model

# Funktion zum Laden des Modells
def load_model(model_path, num_classes=10, num_digits=5):
    model = crnet(num_classes=num_classes, num_digits=num_digits)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()  # Set the model to evaluation mode
    return model

# Funktion zur Vorhersage des Zählerstands
def predict_meter_reading(model, image_path, device='cpu'):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension

    model.to(device)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        predictions = [torch.argmax(output, dim=1).item() for output in outputs]

    return predictions

# Pfade
model_path = '/Users/janriedel/Downloads/detector/crnet_dd.pth'
test_images_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/yolov5_data/test/images'
output_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/predictions.txt'

# Laden des Modells
model = load_model(model_path)

# Vorhersagen für alle Testbilder
results = []
for image_name in os.listdir(test_images_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(test_images_dir, image_name)
        predictions = predict_meter_reading(model, image_path)
        results.append((image_name, predictions))

# Speichern der Ergebnisse
with open(output_file, 'w') as f:
    for image_name, predictions in results:
        f.write(f"{image_name}: {''.join(map(str, predictions))}\n")

print(f"Predictions saved to {output_file}")


In [ ]:
### Sortierung der Predictions 'sort_predictions_test.py'

# Datei zum Lesen und Sortieren der Vorhersagen
input_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/predictions.txt'
output_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/predictions_sorted.txt'

# Funktion zum Extrahieren der Nummer aus dem Dateinamen
def extract_number(filename):
    import re
    match = re.search(r'meter(\d+)', filename)
    return int(match.group(1)) if match else None

# Lesen der Vorhersagen aus der Datei
with open(input_file, 'r') as f:
    lines = f.readlines()

# Sortieren der Vorhersagen nach Dateinummer
sorted_lines = sorted(lines, key=lambda x: extract_number(x.split(':')[0]))

# Schreiben der sortierten Vorhersagen in die Ausgabedatei
with open(output_file, 'w') as f:
    for line in sorted_lines:
        f.write(line)

print(f"Sorted predictions saved to {output_file}")



In [ ]:
### Abgleich der Predictions 'compare_predictions_test.py'

import os
import re

# Pfade definieren
predictions_file = '/Users/janriedel/Downloads/detector/results/predictions_sorted.txt'
labels_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/yolov5_data/test/labels'
output_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/results_check.txt'

# Funktion zum Extrahieren der Nummer aus dem Dateinamen
def extract_number(filename):
    match = re.search(r'meter(\d+)', filename)
    return int(match.group(1)) if match else None

# Funktion zum Lesen der tatsächlichen Labels aus einer Label-Datei
def read_labels(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    labels = [line.split()[1] for line in lines[:5]]  # Nur die ersten 5 Zeilen
    return labels

# Lesen der Vorhersagen aus der Datei
with open(predictions_file, 'r') as f:
    predictions = f.readlines()

total_predictions = 0
correct_predictions = 0

# Ergebnisvergleich und Schreiben in die Ausgabedatei
with open(output_file, 'w') as f:
    for line in predictions:
        image_name, predicted_reading = line.strip().split(': ')
        predicted_digits = list(predicted_reading)

        # Pfad zur entsprechenden Label-Datei
        label_file = os.path.join(labels_dir, image_name.replace('.jpg', '.txt'))

        if os.path.exists(label_file):
            actual_digits = read_labels(label_file)
            result = f"{image_name}: Predicted - {predicted_digits}, Actual - {actual_digits}"

            # Überprüfung der Korrektheit
            if predicted_digits == actual_digits:
                result += " - Correct\n"
                correct_predictions += 1
            else:
                incorrect_digits = [f"digit {i+1}" for i in range(len(predicted_digits)) if predicted_digits[i] != actual_digits[i]]
                result += f" - Incorrect ({', '.join(incorrect_digits)})\n"
            total_predictions += 1
        else:
            result = f"{image_name}: Predicted - {predicted_digits}, Actual - File not found\n"

        f.write(result)

# Berechnung des Prozentsatzes
accuracy_percentage = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0

# Schreiben der Zusammenfassung in die Ausgabedatei
with open(output_file, 'a') as f:  # 'a' für append
    f.write(f"\nTotal correct predictions: {correct_predictions}\n")
    f.write(f"Accuracy: {accuracy_percentage:.2f}%\n")

print(f"Results saved to {output_file}")


## 4.5 Ergebnisse Test CRNET-Trainingsmodell

In [ ]:
### check_results.txt


meter0951.jpg: Predicted - ['1', '2', '7', '2', '8'], Actual - ['1', '2', '7', '2', '8'] - Correct
meter0952.jpg: Predicted - ['3', '3', '8', '8', '5'], Actual - ['3', '3', '8', '8', '5'] - Correct
meter0953.jpg: Predicted - ['1', '3', '0', '2', '5'], Actual - ['1', '9', '0', '2', '5'] - Incorrect (digit 2)
meter0954.jpg: Predicted - ['0', '3', '6', '8', '9'], Actual - ['0', '3', '6', '8', '9'] - Correct
meter0955.jpg: Predicted - ['0', '0', '0', '0', '1'], Actual - ['0', '0', '0', '0', '1'] - Correct
meter0956.jpg: Predicted - ['1', '6', '6', '8', '5'], Actual - ['1', '6', '6', '8', '5'] - Correct
meter0957.jpg: Predicted - ['0', '3', '0', '2', '3'], Actual - ['0', '3', '0', '2', '3'] - Correct
meter0958.jpg: Predicted - ['0', '8', '4', '9', '1'], Actual - ['0', '8', '4', '9', '1'] - Correct
meter0959.jpg: Predicted - ['0', '4', '2', '2', '0'], Actual - ['0', '4', '2', '2', '0'] - Correct
meter0960.jpg: Predicted - ['2', '6', '1', '4', '2'], Actual - ['2', '6', '7', '4', '3'] - Incorrect (digit 3, digit 5)
meter0961.jpg: Predicted - ['0', '7', '3', '5', '3'], Actual - ['0', '7', '3', '5', '3'] - Correct
meter0962.jpg: Predicted - ['1', '2', '8', '4', '0'], Actual - ['1', '2', '8', '4', '0'] - Correct
meter0963.jpg: Predicted - ['1', '3', '1', '8', '4'], Actual - ['1', '3', '7', '8', '6'] - Incorrect (digit 3, digit 5)
meter0964.jpg: Predicted - ['1', '3', '8', '3', '3'], Actual - ['1', '3', '8', '3', '3'] - Correct
meter0965.jpg: Predicted - ['0', '8', '3', '6', '2'], Actual - ['0', '8', '3', '0', '2'] - Incorrect (digit 4)
meter0966.jpg: Predicted - ['1', '6', '1', '7', '8'], Actual - ['1', '6', '1', '7', '9'] - Incorrect (digit 5)
meter0967.jpg: Predicted - ['0', '8', '2', '7', '6'], Actual - ['0', '8', '2', '7', '6'] - Correct
meter0968.jpg: Predicted - ['0', '5', '3', '2', '2'], Actual - ['0', '5', '3', '2', '2'] - Correct
meter0969.jpg: Predicted - ['4', '1', '8', '4', '3'], Actual - ['4', '1', '8', '4', '3'] - Correct
meter0970.jpg: Predicted - ['1', '7', '4', '8', '5'], Actual - ['7', '7', '4', '5', '1'] - Incorrect (digit 1, digit 4, digit 5)
meter0971.jpg: Predicted - ['1', '0', '1', '9', '9'], Actual - ['1', '0', '7', '9', '9'] - Incorrect (digit 3)
meter0972.jpg: Predicted - ['1', '0', '6', '1', '3'], Actual - ['1', '0', '6', '1', '3'] - Correct
meter0973.jpg: Predicted - ['0', '5', '9', '6', '2'], Actual - ['0', '5', '9', '6', '2'] - Correct
meter0974.jpg: Predicted - ['0', '6', '3', '9', '1'], Actual - ['0', '6', '3', '9', '1'] - Correct
meter0975.jpg: Predicted - ['1', '5', '5', '5', '4'], Actual - ['4', '5', '5', '5', '4'] - Incorrect (digit 1)
meter0976.jpg: Predicted - ['0', '0', '0', '0', '3'], Actual - ['0', '0', '9', '0', '3'] - Incorrect (digit 3)
meter0977.jpg: Predicted - ['1', '5', '6', '6', '5'], Actual - ['1', '5', '6', '0', '5'] - Incorrect (digit 4)
meter0978.jpg: Predicted - ['0', '0', '0', '9', '4'], Actual - ['0', '5', '2', '6', '5'] - Incorrect (digit 2, digit 3, digit 4, digit 5)
meter0979.jpg: Predicted - ['0', '0', '6', '8', '9'], Actual - ['0', '0', '8', '8', '9'] - Incorrect (digit 3)
meter0980.jpg: Predicted - ['0', '1', '4', '5', '8'], Actual - ['0', '1', '4', '5', '9'] - Incorrect (digit 5)
meter0981.jpg: Predicted - ['0', '3', '1', '3', '4'], Actual - ['0', '3', '1', '3', '4'] - Correct
meter0982.jpg: Predicted - ['0', '6', '9', '6', '8'], Actual - ['0', '6', '9', '6', '8'] - Correct
meter0983.jpg: Predicted - ['1', '9', '0', '1', '4'], Actual - ['0', '9', '0', '1', '4'] - Incorrect (digit 1)
meter0984.jpg: Predicted - ['1', '9', '1', '8', '6'], Actual - ['0', '9', '1', '6', '5'] - Incorrect (digit 1, digit 4, digit 5)
meter0985.jpg: Predicted - ['0', '0', '1', '9', '6'], Actual - ['0', '0', '1', '9', '4'] - Incorrect (digit 5)
meter0986.jpg: Predicted - ['0', '6', '2', '4', '4'], Actual - ['0', '6', '7', '3', '3'] - Incorrect (digit 3, digit 4, digit 5)
meter0987.jpg: Predicted - ['2', '2', '2', '3', '6'], Actual - ['2', '2', '2', '3', '6'] - Correct
meter0988.jpg: Predicted - ['1', '7', '1', '1', '9'], Actual - ['1', '7', '1', '7', '8'] - Incorrect (digit 4, digit 5)
meter0989.jpg: Predicted - ['0', '5', '3', '4', '6'], Actual - ['0', '5', '3', '4', '6'] - Correct
meter0990.jpg: Predicted - ['1', '4', '6', '6', '2'], Actual - ['1', '4', '6', '6', '2'] - Correct
meter0991.jpg: Predicted - ['1', '4', '3', '7', '4'], Actual - ['1', '4', '3', '7', '4'] - Correct
meter0992.jpg: Predicted - ['2', '4', '9', '5', '1'], Actual - ['2', '4', '9', '5', '1'] - Correct
meter0993.jpg: Predicted - ['1', '0', '3', '0', '3'], Actual - ['1', '0', '3', '0', '3'] - Correct
meter0994.jpg: Predicted - ['1', '3', '1', '7', '8'], Actual - ['1', '3', '1', '7', '8'] - Correct
meter0995.jpg: Predicted - ['0', '0', '0', '0', '4'], Actual - ['4', '0', '0', '8', '0'] - Incorrect (digit 1, digit 4, digit 5)
meter0996.jpg: Predicted - ['0', '7', '1', '9', '6'], Actual - ['0', '1', '1', '9', '5'] - Incorrect (digit 2, digit 5)
meter0997.jpg: Predicted - ['0', '5', '7', '3', '3'], Actual - ['0', '5', '7', '3', '3'] - Correct
meter0998.jpg: Predicted - ['0', '3', '0', '5', '9'], Actual - ['0', '2', '0', '5', '3'] - Incorrect (digit 2, digit 5)
meter0999.jpg: Predicted - ['1', '7', '5', '5', '3'], Actual - ['1', '1', '5', '5', '3'] - Incorrect (digit 2)
meter1000.jpg: Predicted - ['0', '9', '6', '9', '1'], Actual - ['0', '9', '6', '9', '1'] - Correct
meter1001.jpg: Predicted - ['0', '5', '2', '4', '6'], Actual - ['0', '5', '2', '4', '6'] - Correct
meter1002.jpg: Predicted - ['0', '5', '3', '8', '1'], Actual - ['0', '5', '3', '8', '2'] - Incorrect (digit 5)
meter1003.jpg: Predicted - ['0', '5', '2', '4', '6'], Actual - ['0', '5', '2', '4', '6'] - Correct
meter1004.jpg: Predicted - ['1', '0', '2', '1', '1'], Actual - ['1', '0', '2', '1', '3'] - Incorrect (digit 5)
meter1005.jpg: Predicted - ['0', '9', '6', '9', '9'], Actual - ['0', '9', '6', '9', '9'] - Correct
meter1006.jpg: Predicted - ['0', '9', '4', '4', '4'], Actual - ['0', '9', '4', '4', '4'] - Correct
meter1007.jpg: Predicted - ['0', '9', '0', '7', '2'], Actual - ['0', '9', '0', '7', '2'] - Correct
meter1008.jpg: Predicted - ['1', '1', '9', '8', '2'], Actual - ['1', '1', '9', '8', '2'] - Correct
meter1009.jpg: Predicted - ['1', '3', '8', '4', '7'], Actual - ['1', '3', '8', '4', '7'] - Correct
meter1010.jpg: Predicted - ['1', '7', '5', '1', '7'], Actual - ['1', '0', '3', '1', '7'] - Incorrect (digit 2, digit 3)
meter1011.jpg: Predicted - ['2', '5', '1', '3', '5'], Actual - ['2', '2', '1', '3', '4'] - Incorrect (digit 2, digit 5)
meter1012.jpg: Predicted - ['2', '2', '1', '9', '5'], Actual - ['2', '2', '7', '9', '5'] - Incorrect (digit 3)
meter1013.jpg: Predicted - ['1', '1', '4', '6', '1'], Actual - ['3', '1', '4', '5', '1'] - Incorrect (digit 1, digit 4)
meter1014.jpg: Predicted - ['1', '4', '5', '3', '1'], Actual - ['1', '4', '5', '3', '1'] - Correct
meter1015.jpg: Predicted - ['1', '8', '8', '5', '6'], Actual - ['1', '8', '8', '5', '8'] - Incorrect (digit 5)
meter1016.jpg: Predicted - ['1', '0', '1', '2', '8'], Actual - ['1', '0', '1', '3', '9'] - Incorrect (digit 4, digit 5)
meter1017.jpg: Predicted - ['0', '8', '2', '8', '9'], Actual - ['0', '8', '2', '0', '9'] - Incorrect (digit 4)
meter1018.jpg: Predicted - ['4', '7', '6', '1', '4'], Actual - ['4', '7', '6', '0', '5'] - Incorrect (digit 4, digit 5)
meter1019.jpg: Predicted - ['1', '0', '5', '9', '2'], Actual - ['6', '0', '5', '9', '2'] - Incorrect (digit 1)
meter1020.jpg: Predicted - ['3', '1', '6', '5', '4'], Actual - ['3', '1', '6', '5', '7'] - Incorrect (digit 5)
meter1021.jpg: Predicted - ['0', '7', '0', '5', '8'], Actual - ['8', '7', '0', '5', '0'] - Incorrect (digit 1, digit 5)
meter1022.jpg: Predicted - ['2', '3', '3', '2', '2'], Actual - ['2', '3', '3', '2', '2'] - Correct
meter1023.jpg: Predicted - ['0', '7', '8', '6', '1'], Actual - ['0', '7', '8', '6', '7'] - Incorrect (digit 5)
meter1024.jpg: Predicted - ['0', '3', '3', '4', '8'], Actual - ['0', '3', '3', '8', '6'] - Incorrect (digit 4, digit 5)
meter1025.jpg: Predicted - ['1', '0', '9', '4', '6'], Actual - ['1', '0', '9', '4', '6'] - Correct
meter1026.jpg: Predicted - ['1', '9', '9', '2', '0'], Actual - ['1', '9', '9', '2', '0'] - Correct
meter1027.jpg: Predicted - ['0', '6', '0', '6', '0'], Actual - ['0', '6', '0', '6', '0'] - Correct
meter1028.jpg: Predicted - ['1', '4', '2', '0', '6'], Actual - ['1', '4', '2', '0', '6'] - Correct
meter1029.jpg: Predicted - ['0', '9', '2', '8', '3'], Actual - ['0', '3', '3', '8', '3'] - Incorrect (digit 2, digit 3)
meter1030.jpg: Predicted - ['0', '6', '1', '9', '2'], Actual - ['0', '6', '1', '9', '7'] - Incorrect (digit 5)
meter1031.jpg: Predicted - ['0', '6', '5', '6', '9'], Actual - ['0', '6', '5', '6', '9'] - Correct
meter1032.jpg: Predicted - ['1', '4', '2', '7', '9'], Actual - ['1', '4', '2', '7', '9'] - Correct
meter1033.jpg: Predicted - ['1', '0', '9', '2', '2'], Actual - ['1', '0', '9', '2', '2'] - Correct
meter1034.jpg: Predicted - ['1', '1', '7', '7', '5'], Actual - ['1', '1', '7', '7', '5'] - Correct
meter1035.jpg: Predicted - ['1', '4', '3', '7', '5'], Actual - ['1', '4', '3', '7', '5'] - Correct
meter1036.jpg: Predicted - ['1', '0', '9', '3', '5'], Actual - ['2', '0', '9', '3', '5'] - Incorrect (digit 1)
meter1037.jpg: Predicted - ['0', '7', '9', '5', '0'], Actual - ['0', '7', '0', '5', '1'] - Incorrect (digit 3, digit 5)
meter1038.jpg: Predicted - ['1', '7', '4', '6', '1'], Actual - ['1', '7', '4', '6', '1'] - Correct
meter1039.jpg: Predicted - ['0', '5', '7', '2', '9'], Actual - ['0', '5', '7', '2', '9'] - Correct
meter1040.jpg: Predicted - ['0', '9', '9', '3', '1'], Actual - ['0', '9', '9', '3', '1'] - Correct
meter1041.jpg: Predicted - ['0', '5', '6', '2', '5'], Actual - ['0', '9', '6', '3', '5'] - Incorrect (digit 2, digit 4)
meter1042.jpg: Predicted - ['0', '9', '8', '6', '5'], Actual - ['0', '9', '8', '6', '5'] - Correct
meter1043.jpg: Predicted - ['0', '9', '5', '0', '4'], Actual - ['0', '9', '5', '0', '9'] - Incorrect (digit 5)
meter1044.jpg: Predicted - ['0', '8', '9', '7', '9'], Actual - ['0', '8', '9', '7', '9'] - Correct
meter1045.jpg: Predicted - ['0', '6', '0', '2', '8'], Actual - ['0', '8', '0', '2', '6'] - Incorrect (digit 2, digit 5)
meter1046.jpg: Predicted - ['0', '7', '3', '0', '3'], Actual - ['0', '7', '3', '0', '3'] - Correct
meter1047.jpg: Predicted - ['0', '7', '9', '1', '3'], Actual - ['0', '7', '9', '1', '3'] - Correct
meter1048.jpg: Predicted - ['1', '1', '7', '1', '9'], Actual - ['1', '1', '7', '1', '9'] - Correct
meter1049.jpg: Predicted - ['1', '3', '4', '7', '4'], Actual - ['1', '3', '4', '7', '4'] - Correct
meter1050.jpg: Predicted - ['1', '0', '5', '4', '6'], Actual - ['1', '8', '5', '4', '6'] - Incorrect (digit 2)
meter1051.jpg: Predicted - ['0', '4', '7', '7', '5'], Actual - ['0', '4', '7', '7', '5'] - Correct
meter1052.jpg: Predicted - ['0', '8', '9', '5', '6'], Actual - ['0', '8', '9', '5', '6'] - Correct
meter1053.jpg: Predicted - ['0', '0', '9', '8', '3'], Actual - ['0', '8', '9', '0', '3'] - Incorrect (digit 2, digit 4)
meter1054.jpg: Predicted - ['1', '2', '0', '1', '8'], Actual - ['1', '2', '0', '1', '8'] - Correct
meter1055.jpg: Predicted - ['0', '4', '1', '7', '8'], Actual - ['0', '4', '1', '7', '8'] - Correct
meter1056.jpg: Predicted - ['0', '4', '1', '8', '4'], Actual - ['0', '4', '1', '8', '4'] - Correct
meter1057.jpg: Predicted - ['0', '3', '9', '3', '1'], Actual - ['0', '3', '9', '3', '1'] - Correct
meter1058.jpg: Predicted - ['1', '4', '4', '4', '3'], Actual - ['1', '4', '4', '4', '3'] - Correct
meter1059.jpg: Predicted - ['1', '0', '0', '9', '3'], Actual - ['1', '0', '0', '9', '3'] - Correct
meter1060.jpg: Predicted - ['0', '3', '5', '0', '2'], Actual - ['0', '3', '3', '0', '2'] - Incorrect (digit 3)
meter1061.jpg: Predicted - ['0', '0', '5', '3', '8'], Actual - ['0', '0', '5', '3', '8'] - Correct
meter1062.jpg: Predicted - ['0', '8', '6', '4', '5'], Actual - ['0', '8', '6', '4', '5'] - Correct
meter1063.jpg: Predicted - ['0', '5', '6', '5', '4'], Actual - ['0', '5', '6', '5', '4'] - Correct
meter1064.jpg: Predicted - ['1', '3', '3', '7', '5'], Actual - ['1', '3', '3', '7', '5'] - Correct
meter1065.jpg: Predicted - ['1', '2', '3', '2', '7'], Actual - ['1', '2', '3', '2', '6'] - Incorrect (digit 5)
meter1066.jpg: Predicted - ['0', '6', '0', '4', '4'], Actual - ['0', '6', '0', '4', '4'] - Correct
meter1067.jpg: Predicted - ['0', '4', '4', '8', '8'], Actual - ['0', '4', '4', '6', '8'] - Incorrect (digit 4)
meter1068.jpg: Predicted - ['0', '7', '7', '9', '4'], Actual - ['0', '7', '7', '9', '2'] - Incorrect (digit 5)
meter1069.jpg: Predicted - ['5', '7', '7', '3', '3'], Actual - ['5', '7', '7', '3', '3'] - Correct
meter1070.jpg: Predicted - ['0', '0', '0', '6', '3'], Actual - ['5', '8', '3', '6', '3'] - Incorrect (digit 1, digit 2, digit 3)
meter1071.jpg: Predicted - ['2', '0', '3', '2', '5'], Actual - ['2', '0', '3', '2', '5'] - Correct
meter1072.jpg: Predicted - ['0', '8', '8', '5', '6'], Actual - ['0', '8', '8', '5', '4'] - Incorrect (digit 5)
meter1073.jpg: Predicted - ['0', '4', '1', '5', '7'], Actual - ['0', '4', '1', '5', '7'] - Correct
meter1074.jpg: Predicted - ['0', '1', '6', '5', '6'], Actual - ['0', '3', '6', '5', '6'] - Incorrect (digit 2)
meter1075.jpg: Predicted - ['0', '1', '9', '5', '7'], Actual - ['0', '1', '9', '5', '7'] - Correct
meter1076.jpg: Predicted - ['3', '6', '9', '6', '3'], Actual - ['3', '6', '9', '6', '3'] - Correct
meter1077.jpg: Predicted - ['0', '2', '3', '6', '3'], Actual - ['0', '8', '5', '5', '1'] - Incorrect (digit 2, digit 3, digit 4, digit 5)
meter1078.jpg: Predicted - ['3', '2', '7', '8', '3'], Actual - ['3', '2', '7', '8', '3'] - Correct
meter1079.jpg: Predicted - ['0', '6', '0', '0', '5'], Actual - ['0', '6', '0', '0', '5'] - Correct
meter1080.jpg: Predicted - ['2', '1', '7', '0', '2'], Actual - ['2', '1', '7', '0', '2'] - Correct
meter1081.jpg: Predicted - ['1', '3', '2', '0', '6'], Actual - ['1', '4', '2', '0', '6'] - Incorrect (digit 2)
meter1082.jpg: Predicted - ['1', '6', '8', '1', '6'], Actual - ['1', '6', '8', '1', '6'] - Correct
meter1083.jpg: Predicted - ['0', '3', '9', '5', '7'], Actual - ['0', '3', '9', '5', '7'] - Correct
meter1084.jpg: Predicted - ['1', '9', '1', '5', '2'], Actual - ['1', '9', '1', '5', '2'] - Correct
meter1085.jpg: Predicted - ['0', '1', '2', '7', '1'], Actual - ['0', '1', '2', '7', '1'] - Correct
meter1086.jpg: Predicted - ['1', '7', '7', '4', '6'], Actual - ['0', '7', '7', '4', '6'] - Incorrect (digit 1)
meter1087.jpg: Predicted - ['0', '8', '6', '7', '3'], Actual - ['0', '8', '6', '7', '3'] - Correct
meter1088.jpg: Predicted - ['1', '3', '6', '5', '2'], Actual - ['1', '3', '6', '5', '2'] - Correct
meter1089.jpg: Predicted - ['0', '5', '4', '9', '0'], Actual - ['0', '5', '4', '9', '0'] - Correct
meter1090.jpg: Predicted - ['1', '6', '3', '4', '8'], Actual - ['1', '6', '3', '4', '8'] - Correct
meter1091.jpg: Predicted - ['1', '4', '9', '5', '5'], Actual - ['1', '4', '9', '5', '5'] - Correct
meter1092.jpg: Predicted - ['0', '8', '8', '1', '6'], Actual - ['0', '8', '8', '1', '6'] - Correct
meter1093.jpg: Predicted - ['0', '7', '3', '6', '3'], Actual - ['0', '7', '3', '6', '3'] - Correct
meter1094.jpg: Predicted - ['0', '9', '5', '0', '9'], Actual - ['0', '9', '5', '0', '9'] - Correct
meter1095.jpg: Predicted - ['1', '2', '5', '7', '4'], Actual - ['1', '2', '5', '2', '4'] - Incorrect (digit 4)
meter1096.jpg: Predicted - ['1', '0', '0', '6', '8'], Actual - ['1', '3', '0', '6', '8'] - Incorrect (digit 2)
meter1097.jpg: Predicted - ['0', '8', '9', '5', '6'], Actual - ['0', '8', '9', '5', '6'] - Correct
meter1098.jpg: Predicted - ['0', '6', '3', '2', '8'], Actual - ['0', '6', '3', '2', '8'] - Correct
meter1099.jpg: Predicted - ['0', '0', '0', '7', '9'], Actual - ['0', '8', '0', '7', '9'] - Incorrect (digit 2)
meter1100.jpg: Predicted - ['1', '0', '3', '1', '7'], Actual - ['1', '0', '3', '1', '7'] - Correct
meter1101.jpg: Predicted - ['2', '4', '2', '1', '7'], Actual - ['2', '4', '2', '1', '7'] - Correct
meter1102.jpg: Predicted - ['1', '7', '1', '5', '9'], Actual - ['1', '7', '1', '5', '9'] - Correct
meter1103.jpg: Predicted - ['2', '5', '8', '9', '9'], Actual - ['2', '5', '8', '9', '9'] - Correct
meter1104.jpg: Predicted - ['0', '8', '0', '9', '6'], Actual - ['0', '8', '0', '9', '6'] - Correct
meter1105.jpg: Predicted - ['0', '5', '4', '4', '4'], Actual - ['0', '5', '4', '4', '4'] - Correct
meter1106.jpg: Predicted - ['2', '2', '2', '3', '6'], Actual - ['2', '2', '2', '3', '6'] - Correct
meter1107.jpg: Predicted - ['2', '9', '7', '2', '1'], Actual - ['2', '9', '7', '2', '1'] - Correct
meter1108.jpg: Predicted - ['6', '2', '1', '3', '4'], Actual - ['9', '2', '1', '3', '4'] - Incorrect (digit 1)
meter1109.jpg: Predicted - ['2', '1', '7', '6', '5'], Actual - ['2', '1', '7', '6', '5'] - Correct
meter1110.jpg: Predicted - ['1', '2', '3', '6', '1'], Actual - ['1', '2', '3', '6', '1'] - Correct
meter1111.jpg: Predicted - ['2', '7', '6', '0', '3'], Actual - ['2', '7', '6', '0', '3'] - Correct
meter1112.jpg: Predicted - ['0', '4', '5', '9', '4'], Actual - ['6', '4', '5', '9', '4'] - Incorrect (digit 1)
meter1113.jpg: Predicted - ['1', '4', '9', '8', '7'], Actual - ['1', '4', '9', '3', '7'] - Incorrect (digit 4)
meter1114.jpg: Predicted - ['0', '5', '4', '5', '5'], Actual - ['0', '5', '4', '5', '5'] - Correct
meter1115.jpg: Predicted - ['0', '3', '4', '1', '1'], Actual - ['0', '3', '4', '1', '1'] - Correct
meter1116.jpg: Predicted - ['2', '6', '4', '4', '5'], Actual - ['2', '6', '4', '4', '5'] - Correct
meter1117.jpg: Predicted - ['0', '1', '9', '3', '5'], Actual - ['0', '1', '9', '3', '5'] - Correct
meter1118.jpg: Predicted - ['1', '2', '2', '7', '0'], Actual - ['1', '2', '2', '7', '0'] - Correct
meter1119.jpg: Predicted - ['0', '0', '7', '0', '6'], Actual - ['0', '0', '7', '0', '6'] - Correct
meter1120.jpg: Predicted - ['0', '5', '5', '2', '8'], Actual - ['0', '5', '5', '2', '8'] - Correct
meter1121.jpg: Predicted - ['0', '3', '5', '4', '8'], Actual - ['0', '3', '5', '4', '6'] - Incorrect (digit 5)
meter1122.jpg: Predicted - ['1', '4', '6', '0', '7'], Actual - ['1', '4', '6', '0', '7'] - Correct
meter1123.jpg: Predicted - ['2', '2', '7', '5', '7'], Actual - ['2', '2', '7', '5', '7'] - Correct
meter1124.jpg: Predicted - ['0', '3', '2', '8', '8'], Actual - ['0', '3', '2', '8', '8'] - Correct
meter1125.jpg: Predicted - ['4', '6', '5', '8', '8'], Actual - ['4', '4', '5', '8', '9'] - Incorrect (digit 2, digit 5)
meter1126.jpg: Predicted - ['2', '7', '6', '4', '1'], Actual - ['2', '7', '6', '4', '1'] - Correct
meter1127.jpg: Predicted - ['0', '5', '2', '4', '2'], Actual - ['0', '5', '2', '4', '2'] - Correct
meter1128.jpg: Predicted - ['1', '0', '3', '9', '1'], Actual - ['1', '0', '3', '9', '1'] - Correct
meter1129.jpg: Predicted - ['4', '3', '8', '3', '3'], Actual - ['4', '3', '8', '3', '3'] - Correct
meter1130.jpg: Predicted - ['0', '1', '1', '2', '5'], Actual - ['0', '1', '1', '2', '5'] - Correct
meter1131.jpg: Predicted - ['3', '3', '1', '9', '0'], Actual - ['3', '3', '1', '9', '0'] - Correct
meter1132.jpg: Predicted - ['1', '1', '1', '2', '7'], Actual - ['1', '1', '1', '2', '1'] - Incorrect (digit 5)
meter1133.jpg: Predicted - ['0', '0', '5', '9', '8'], Actual - ['8', '9', '5', '9', '8'] - Incorrect (digit 1, digit 2)
meter1134.jpg: Predicted - ['0', '3', '8', '6', '8'], Actual - ['0', '3', '8', '6', '8'] - Correct
meter1135.jpg: Predicted - ['0', '4', '9', '2', '6'], Actual - ['0', '4', '9', '2', '6'] - Correct
meter1136.jpg: Predicted - ['2', '6', '3', '7', '1'], Actual - ['1', '8', '3', '7', '1'] - Incorrect (digit 1, digit 2)
meter1137.jpg: Predicted - ['1', '6', '5', '9', '0'], Actual - ['1', '6', '5', '9', '0'] - Correct
meter1138.jpg: Predicted - ['0', '5', '8', '3', '8'], Actual - ['9', '5', '8', '3', '0'] - Incorrect (digit 1, digit 5)
meter1139.jpg: Predicted - ['2', '4', '2', '1', '2'], Actual - ['2', '4', '2', '1', '2'] - Correct
meter1140.jpg: Predicted - ['5', '0', '6', '8', '9'], Actual - ['5', '0', '6', '8', '9'] - Correct
meter1141.jpg: Predicted - ['0', '2', '2', '9', '0'], Actual - ['4', '2', '2', '4', '0'] - Incorrect (digit 1, digit 4)
meter1142.jpg: Predicted - ['0', '9', '7', '2', '6'], Actual - ['6', '9', '7', '2', '6'] - Incorrect (digit 1)
meter1143.jpg: Predicted - ['4', '7', '8', '1', '4'], Actual - ['4', '8', '8', '1', '0'] - Incorrect (digit 2, digit 5)
meter1144.jpg: Predicted - ['0', '3', '1', '4', '4'], Actual - ['0', '3', '1', '4', '4'] - Correct
meter1145.jpg: Predicted - ['0', '8', '1', '6', '6'], Actual - ['0', '8', '1', '0', '6'] - Incorrect (digit 4)
meter1146.jpg: Predicted - ['1', '8', '5', '9', '5'], Actual - ['7', '8', '5', '9', '5'] - Incorrect (digit 1)
meter1147.jpg: Predicted - ['2', '1', '1', '2', '1'], Actual - ['2', '1', '1', '2', '1'] - Correct
meter1148.jpg: Predicted - ['0', '6', '1', '5', '1'], Actual - ['6', '6', '1', '5', '1'] - Incorrect (digit 1)
meter1149.jpg: Predicted - ['1', '2', '3', '2', '5'], Actual - ['1', '2', '3', '2', '5'] - Correct
meter1150.jpg: Predicted - ['0', '3', '9', '8', '6'], Actual - ['0', '3', '9', '8', '6'] - Correct
meter1151.jpg: Predicted - ['4', '8', '1', '4', '8'], Actual - ['4', '8', '1', '4', '9'] - Incorrect (digit 5)
meter1152.jpg: Predicted - ['0', '6', '9', '1', '8'], Actual - ['0', '6', '9', '1', '8'] - Correct
meter1153.jpg: Predicted - ['3', '5', '2', '7', '0'], Actual - ['7', '5', '2', '7', '0'] - Incorrect (digit 1)
meter1154.jpg: Predicted - ['1', '0', '8', '8', '1'], Actual - ['1', '6', '8', '0', '0'] - Incorrect (digit 2, digit 4, digit 5)
meter1155.jpg: Predicted - ['3', '9', '9', '4', '8'], Actual - ['9', '9', '9', '4', '0'] - Incorrect (digit 1, digit 5)
meter1156.jpg: Predicted - ['0', '7', '5', '1', '1'], Actual - ['0', '7', '5', '1', '1'] - Correct
meter1157.jpg: Predicted - ['1', '0', '0', '0', '1'], Actual - ['6', '0', '0', '9', '1'] - Incorrect (digit 1, digit 4)
meter1158.jpg: Predicted - ['1', '5', '6', '0', '9'], Actual - ['4', '5', '6', '0', '9'] - Incorrect (digit 1)
meter1159.jpg: Predicted - ['1', '2', '6', '3', '0'], Actual - ['1', '2', '6', '3', '0'] - Correct
meter1160.jpg: Predicted - ['6', '7', '5', '4', '8'], Actual - ['6', '1', '5', '4', '8'] - Incorrect (digit 2)
meter1161.jpg: Predicted - ['3', '4', '1', '7', '9'], Actual - ['3', '4', '1', '7', '6'] - Incorrect (digit 5)
meter1162.jpg: Predicted - ['0', '4', '9', '3', '8'], Actual - ['0', '4', '9', '3', '8'] - Correct
meter1163.jpg: Predicted - ['0', '6', '1', '5', '8'], Actual - ['0', '6', '1', '5', '8'] - Correct
meter1164.jpg: Predicted - ['1', '3', '4', '9', '0'], Actual - ['1', '3', '4', '9', '0'] - Correct
meter1165.jpg: Predicted - ['1', '7', '2', '2', '7'], Actual - ['1', '7', '2', '2', '7'] - Correct
meter1166.jpg: Predicted - ['0', '9', '9', '3', '1'], Actual - ['0', '9', '9', '3', '1'] - Correct
meter1167.jpg: Predicted - ['0', '6', '9', '4', '7'], Actual - ['0', '6', '9', '4', '2'] - Incorrect (digit 5)
meter1168.jpg: Predicted - ['0', '8', '4', '1', '2'], Actual - ['0', '8', '4', '1', '2'] - Correct
meter1169.jpg: Predicted - ['0', '5', '7', '2', '9'], Actual - ['0', '5', '7', '2', '9'] - Correct
meter1170.jpg: Predicted - ['1', '2', '3', '5', '3'], Actual - ['1', '2', '3', '5', '3'] - Correct
meter1171.jpg: Predicted - ['0', '4', '4', '2', '5'], Actual - ['0', '4', '4', '2', '5'] - Correct
meter1172.jpg: Predicted - ['1', '0', '5', '4', '7'], Actual - ['1', '0', '5', '4', '7'] - Correct
meter1173.jpg: Predicted - ['0', '7', '0', '5', '1'], Actual - ['0', '7', '0', '5', '1'] - Correct
meter1174.jpg: Predicted - ['0', '6', '2', '1', '0'], Actual - ['0', '6', '2', '1', '0'] - Correct
meter1175.jpg: Predicted - ['1', '1', '4', '9', '6'], Actual - ['1', '1', '4', '9', '6'] - Correct
meter1176.jpg: Predicted - ['0', '8', '0', '2', '6'], Actual - ['0', '8', '0', '2', '6'] - Correct
meter1177.jpg: Predicted - ['1', '0', '0', '9', '7'], Actual - ['1', '0', '0', '9', '3'] - Incorrect (digit 5)
meter1178.jpg: Predicted - ['1', '4', '4', '2', '1'], Actual - ['1', '4', '4', '2', '5'] - Incorrect (digit 5)
meter1179.jpg: Predicted - ['1', '2', '8', '3', '7'], Actual - ['1', '2', '8', '3', '7'] - Correct
meter1180.jpg: Predicted - ['1', '0', '4', '1', '2'], Actual - ['1', '0', '4', '1', '2'] - Correct
meter1181.jpg: Predicted - ['0', '8', '7', '6', '4'], Actual - ['0', '8', '7', '6', '4'] - Correct
meter1182.jpg: Predicted - ['1', '3', '0', '0', '1'], Actual - ['1', '4', '0', '0', '1'] - Incorrect (digit 2)
meter1183.jpg: Predicted - ['0', '8', '9', '8', '3'], Actual - ['0', '8', '9', '0', '3'] - Incorrect (digit 4)
meter1184.jpg: Predicted - ['0', '4', '1', '7', '8'], Actual - ['0', '4', '1', '7', '8'] - Correct
meter1185.jpg: Predicted - ['1', '4', '9', '7', '9'], Actual - ['1', '4', '9', '7', '9'] - Correct
meter1186.jpg: Predicted - ['0', '0', '0', '0', '0'], Actual - ['0', '0', '0', '0', '0'] - Correct
meter1187.jpg: Predicted - ['0', '9', '8', '3', '9'], Actual - ['0', '9', '8', '3', '9'] - Correct
meter1188.jpg: Predicted - ['0', '2', '7', '9', '1'], Actual - ['0', '2', '7', '9', '1'] - Correct
meter1189.jpg: Predicted - ['0', '2', '2', '2', '1'], Actual - ['0', '3', '2', '2', '3'] - Incorrect (digit 2, digit 5)
meter1190.jpg: Predicted - ['0', '8', '7', '8', '8'], Actual - ['0', '1', '7', '8', '8'] - Incorrect (digit 2)
meter1191.jpg: Predicted - ['2', '0', '4', '6', '9'], Actual - ['2', '0', '4', '6', '9'] - Correct
meter1192.jpg: Predicted - ['1', '3', '0', '0', '4'], Actual - ['1', '3', '0', '0', '4'] - Correct
meter1193.jpg: Predicted - ['0', '5', '0', '1', '8'], Actual - ['0', '6', '0', '1', '8'] - Incorrect (digit 2)
meter1194.jpg: Predicted - ['1', '8', '3', '9', '5'], Actual - ['1', '8', '3', '9', '5'] - Correct
meter1195.jpg: Predicted - ['0', '5', '5', '2', '5'], Actual - ['0', '5', '5', '2', '5'] - Correct
meter1196.jpg: Predicted - ['3', '7', '1', '7', '8'], Actual - ['3', '7', '1', '7', '0'] - Incorrect (digit 5)
meter1197.jpg: Predicted - ['0', '6', '6', '7', '3'], Actual - ['0', '6', '6', '7', '3'] - Correct
meter1198.jpg: Predicted - ['0', '1', '5', '2', '1'], Actual - ['0', '1', '5', '2', '1'] - Correct
meter1199.jpg: Predicted - ['2', '2', '2', '4', '7'], Actual - ['2', '2', '2', '4', '7'] - Correct
meter1200.jpg: Predicted - ['0', '7', '3', '1', '7'], Actual - ['0', '7', '3', '1', '7'] - Correct

Total correct predictions: 159
Accuracy: 63.60%


Diese Textdatei wurde anhand des folgenden Skriptes um neue Berechnungen ergänzt, die sich auf die Predictions für die einzelnen Ziffern beziehen.

In [ ]:
# Datei-Pfad
file_path = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/results_check.txt'

# Variablen initialisieren
total_digits = 0
correct_digits = 0
digit_correct_count = [0, 0, 0, 0, 0]
digit_total_count = [0, 0, 0, 0, 0]

# Datei einlesen und Werte extrahieren
with open(file_path, 'r') as file:
    lines = file.readlines()

# Daten verarbeiten
for line in lines:
    if "Predicted" in line and "Actual" in line:
        predicted = line.strip().split("Predicted - [")[1].split("]")[0].split(", ")
        actual = line.strip().split("Actual - [")[1].split("]")[0].split(", ")
        for i in range(5):
            digit_total_count[i] += 1
            if predicted[i] == actual[i]:
                correct_digits += 1
                digit_correct_count[i] += 1
        total_digits += 5

# Gesamtanzahl korrekt erkannter Ziffern
overall_accuracy = (correct_digits / total_digits) * 100

# Prozentsatz der korrekt erkannten Ziffern pro Ziffernposition
digit_accuracy = [(count / digit_total_count[i]) * 100 for i, count in enumerate(digit_correct_count)]

# Ergebnisse zur Datei hinzufügen
with open(file_path, 'a') as file:
    file.write(f"\nTotal correct digits: {correct_digits}\n")
    file.write(f"Overall digit accuracy: {overall_accuracy:.2f}%\n")
    for i, accuracy in enumerate(digit_accuracy):
        file.write(f"Digit {i+1} accuracy: {accuracy:.2f}%\n")

    # Prozentsatz der insgesamt richtig erkannten Ziffern berechnen
    total_readings = len([line for line in lines if "Predicted" in line and "Actual" in line])
    correct_readings = sum(1 for line in lines if "Predicted" in line and "Actual" in line and line.strip().split("Predicted - [")[1].split("]")[0].split(", ") == line.strip().split("Actual - [")[1].split("]")[0].split(", "))
    total_digit_accuracy = (correct_digits / (total_readings * 5)) * 100

    file.write(f"Total digit accuracy across all readings: {total_digit_accuracy:.2f}%\n")

print("Evaluation complete. Results added to the file.")


In [ ]:
### Ergänzung Digit Predictions

Total correct predictions: 159
Accuracy: 63.60%

Total correct digits: 1115
Overall digit accuracy: 89.20%
Digit 1 accuracy: 90.40%
Digit 2 accuracy: 88.80%
Digit 3 accuracy: 94.40%
Digit 4 accuracy: 90.40%
Digit 5 accuracy: 82.00%
Total digit accuracy across all readings: 89.20%


## 4.6 Test mit Predicted Images
Um die beiden trainierten Modelle (Yolov5 & CRNET) zu verbinden, wurde händisch der neue Ordner 'test_crp' erstellt, der die vom YoloV5-Modell vorhergesagten Counter in 'images' und den Zählerstand als Textdateien in 'labels' enthält. Anschließend wurden die Prozessschritte analog 4.4 & 4.5 durchgeführt.


In [ ]:
### prediction_test.py

import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Definiere die CRNET-Klasse und die crnet-Funktion
class CRNET(nn.Module):
    def __init__(self, num_classes=10, num_digits=5):
        super(CRNET, self).__init__()
        self.num_digits = num_digits
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifiers = nn.ModuleList([nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        ) for _ in range(num_digits)])

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        outputs = [classifier(x) for classifier in self.classifiers]
        return outputs

def crnet(num_classes=10, num_digits=5):
    model = CRNET(num_classes=num_classes, num_digits=num_digits)
    return model

# Funktion zum Laden des Modells
def load_model(model_path, num_classes=10, num_digits=5):
    model = crnet(num_classes=num_classes, num_digits=num_digits)
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()  # Set the model to evaluation mode
    return model

# Funktion zur Vorhersage des Zählerstands
def predict_meter_reading(model, image_path, device='cpu'):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension

    model.to(device)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        predictions = [torch.argmax(output, dim=1).item() for output in outputs]

    return predictions

# Pfade
model_path = '/Users/janriedel/Downloads/detector/crnet_dd.pth'
test_images_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/test_crp/images'
output_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/predictions_crp.txt'

# Laden des Modells
model = load_model(model_path)

# Vorhersagen für alle Testbilder
results = []
for image_name in os.listdir(test_images_dir):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(test_images_dir, image_name)
        predictions = predict_meter_reading(model, image_path)
        results.append((image_name, predictions))

# Speichern der Ergebnisse
with open(output_file, 'w') as f:
    for image_name, predictions in results:
        f.write(f"{image_name}: {''.join(map(str, predictions))}\n")

print(f"Predictions saved to {output_file}")


In [ ]:
### sort_predictions_test.py

# Datei zum Lesen und Sortieren der Vorhersagen
input_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/predictions_crp.txt'
output_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/predictions_sorted_crp.txt'

# Funktion zum Extrahieren der Nummer aus dem Dateinamen
def extract_number(filename):
    import re
    match = re.search(r'meter(\d+)', filename)
    return int(match.group(1)) if match else None

# Lesen der Vorhersagen aus der Datei
with open(input_file, 'r') as f:
    lines = f.readlines()

# Sortieren der Vorhersagen nach Dateinummer
sorted_lines = sorted(lines, key=lambda x: extract_number(x.split(':')[0]))

# Schreiben der sortierten Vorhersagen in die Ausgabedatei
with open(output_file, 'w') as f:
    for line in sorted_lines:
        f.write(line)

print(f"Sorted predictions saved to {output_file}")


In [ ]:
### compare_predictions_test.py

import os
import re

# Pfade definieren
predictions_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/predictions_sorted_crp.txt'
labels_dir = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/test_crp/labels'
output_file = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/results_check_crp.txt'

# Funktion zum Extrahieren der Nummer aus dem Dateinamen
def extract_number(filename):
    match = re.search(r'meter(\d+)', filename)
    return int(match.group(1)) if match else None

# Funktion zum Lesen der tatsächlichen Labels aus einer Label-Datei
def read_labels(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    labels = [line.split()[1] for line in lines[:5]]  # Nur die ersten 5 Zeilen
    return labels

# Lesen der Vorhersagen aus der Datei
with open(predictions_file, 'r') as f:
    predictions = f.readlines()

total_predictions = 0
correct_predictions = 0

# Ergebnisvergleich und Schreiben in die Ausgabedatei
with open(output_file, 'w') as f:
    for line in predictions:
        image_name, predicted_reading = line.strip().split(': ')
        predicted_digits = list(predicted_reading)

        # Pfad zur entsprechenden Label-Datei
        label_file = os.path.join(labels_dir, image_name.replace('.jpg', '.txt'))

        if os.path.exists(label_file):
            actual_digits = read_labels(label_file)
            result = f"{image_name}: Predicted - {predicted_digits}, Actual - {actual_digits}"

            # Überprüfung der Korrektheit
            if predicted_digits == actual_digits:
                result += " - Correct\n"
                correct_predictions += 1
            else:
                incorrect_digits = [f"digit {i+1}" for i in range(len(predicted_digits)) if predicted_digits[i] != actual_digits[i]]
                result += f" - Incorrect ({', '.join(incorrect_digits)})\n"
            total_predictions += 1
        else:
            result = f"{image_name}: Predicted - {predicted_digits}, Actual - File not found\n"

        f.write(result)

# Berechnung des Prozentsatzes
accuracy_percentage = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0

# Schreiben der Zusammenfassung in die Ausgabedatei
with open(output_file, 'a') as f:  # 'a' für append
    f.write(f"\nTotal correct predictions: {correct_predictions}\n")
    f.write(f"Accuracy: {accuracy_percentage:.2f}%\n")

print(f"Results saved to {output_file}")


In [ ]:
### CRNet results_new_calculations_crp.py

# Datei-Pfad
file_path = '/Users/janriedel/Downloads/detector/CRNET_digit_detection/results/results_check_crp.txt'

# Variablen initialisieren
total_digits = 0
correct_digits = 0
digit_correct_count = [0, 0, 0, 0, 0]
digit_total_count = [0, 0, 0, 0, 0]

# Datei einlesen und Werte extrahieren
with open(file_path, 'r') as file:
    lines = file.readlines()

# Daten verarbeiten
for line in lines:
    if "Predicted" in line and "Actual" in line:
        predicted = line.strip().split("Predicted - [")[1].split("]")[0].split(", ")
        actual = line.strip().split("Actual - [")[1].split("]")[0].split(", ")
        for i in range(5):
            digit_total_count[i] += 1
            if predicted[i] == actual[i]:
                correct_digits += 1
                digit_correct_count[i] += 1
        total_digits += 5

# Gesamtanzahl korrekt erkannter Ziffern
overall_accuracy = (correct_digits / total_digits) * 100

# Prozentsatz der korrekt erkannten Ziffern pro Ziffernposition
digit_accuracy = [(count / digit_total_count[i]) * 100 for i, count in enumerate(digit_correct_count)]

# Ergebnisse zur Datei hinzufügen
with open(file_path, 'a') as file:
    file.write(f"\nTotal correct digits: {correct_digits}\n")
    file.write(f"Overall digit accuracy: {overall_accuracy:.2f}%\n")
    for i, accuracy in enumerate(digit_accuracy):
        file.write(f"Digit {i+1} accuracy: {accuracy:.2f}%\n")

    # Prozentsatz der insgesamt richtig erkannten Ziffern berechnen
    total_readings = len([line for line in lines if "Predicted" in line and "Actual" in line])
    correct_readings = sum(1 for line in lines if "Predicted" in line and "Actual" in line and line.strip().split("Predicted - [")[1].split("]")[0].split(", ") == line.strip().split("Actual - [")[1].split("]")[0].split(", "))
    total_digit_accuracy = (correct_digits / (total_readings * 5)) * 100

    file.write(f"Total digit accuracy across all readings: {total_digit_accuracy:.2f}%\n")

print("Evaluation complete. Results added to the file.")


In [ ]:
### check_results_crp.txt


meter0951.jpg: Predicted - ['1', '2', '7', '2', '8'], Actual - ['1', '2', '7', '2', '8'] - Correct
meter0952.jpg: Predicted - ['1', '3', '8', '8', '5'], Actual - ['3', '3', '8', '8', '5'] - Incorrect (digit 1)
meter0953.jpg: Predicted - ['1', '9', '0', '2', '5'], Actual - ['1', '9', '0', '2', '5'] - Correct
meter0954.jpg: Predicted - ['0', '3', '6', '8', '9'], Actual - ['0', '3', '6', '8', '9'] - Correct
meter0955.jpg: Predicted - ['0', '0', '0', '0', '1'], Actual - ['0', '0', '0', '0', '1'] - Correct
meter0956.jpg: Predicted - ['1', '6', '6', '8', '5'], Actual - ['1', '6', '6', '8', '5'] - Correct
meter0957.jpg: Predicted - ['0', '3', '0', '2', '3'], Actual - ['0', '3', '0', '2', '3'] - Correct
meter0958.jpg: Predicted - ['0', '8', '4', '9', '1'], Actual - ['0', '8', '4', '9', '1'] - Correct
meter0959.jpg: Predicted - ['0', '4', '2', '2', '0'], Actual - ['0', '4', '2', '2', '0'] - Correct
meter0960.jpg: Predicted - ['2', '6', '1', '4', '5'], Actual - ['2', '6', '7', '4', '3'] - Incorrect (digit 3, digit 5)
meter0961.jpg: Predicted - ['0', '7', '3', '5', '3'], Actual - ['0', '7', '3', '5', '3'] - Correct
meter0962.jpg: Predicted - ['1', '2', '8', '4', '0'], Actual - ['1', '2', '8', '4', '0'] - Correct
meter0963.jpg: Predicted - ['1', '3', '1', '8', '4'], Actual - ['1', '3', '7', '8', '6'] - Incorrect (digit 3, digit 5)
meter0964.jpg: Predicted - ['1', '3', '8', '3', '3'], Actual - ['1', '3', '8', '3', '3'] - Correct
meter0965.jpg: Predicted - ['0', '8', '3', '8', '2'], Actual - ['0', '8', '3', '0', '2'] - Incorrect (digit 4)
meter0966.jpg: Predicted - ['1', '6', '1', '7', '8'], Actual - ['1', '6', '1', '7', '9'] - Incorrect (digit 5)
meter0967.jpg: Predicted - ['0', '8', '2', '7', '6'], Actual - ['0', '8', '2', '7', '6'] - Correct
meter0968.jpg: Predicted - ['0', '5', '3', '2', '2'], Actual - ['0', '5', '3', '2', '2'] - Correct
meter0969.jpg: Predicted - ['4', '1', '8', '4', '3'], Actual - ['4', '1', '8', '4', '3'] - Correct
meter0970.jpg: Predicted - ['1', '7', '4', '5', '1'], Actual - ['7', '7', '4', '5', '1'] - Incorrect (digit 1)
meter0971.jpg: Predicted - ['1', '0', '1', '9', '9'], Actual - ['1', '0', '7', '9', '9'] - Incorrect (digit 3)
meter0972.jpg: Predicted - ['1', '0', '6', '1', '3'], Actual - ['1', '0', '6', '1', '3'] - Correct
meter0973.jpg: Predicted - ['0', '5', '9', '6', '2'], Actual - ['0', '5', '9', '6', '2'] - Correct
meter0974.jpg: Predicted - ['0', '6', '6', '9', '1'], Actual - ['0', '6', '3', '9', '1'] - Incorrect (digit 3)
meter0975.jpg: Predicted - ['0', '5', '5', '5', '4'], Actual - ['4', '5', '5', '5', '4'] - Incorrect (digit 1)
meter0976.jpg: Predicted - ['0', '0', '0', '0', '3'], Actual - ['0', '0', '9', '0', '3'] - Incorrect (digit 3)
meter0977.jpg: Predicted - ['1', '5', '0', '6', '5'], Actual - ['1', '5', '6', '0', '5'] - Incorrect (digit 3, digit 4)
meter0978.jpg: Predicted - ['0', '1', '7', '9', '4'], Actual - ['0', '5', '2', '6', '5'] - Incorrect (digit 2, digit 3, digit 4, digit 5)
meter0979.jpg: Predicted - ['0', '0', '6', '8', '9'], Actual - ['0', '0', '8', '8', '9'] - Incorrect (digit 3)
meter0980.jpg: Predicted - ['0', '1', '4', '5', '9'], Actual - ['0', '1', '4', '5', '9'] - Correct
meter0981.jpg: Predicted - ['0', '3', '1', '3', '4'], Actual - ['0', '3', '1', '3', '4'] - Correct
meter0982.jpg: Predicted - ['0', '6', '9', '6', '8'], Actual - ['0', '6', '9', '6', '8'] - Correct
meter0983.jpg: Predicted - ['1', '4', '0', '1', '4'], Actual - ['0', '9', '0', '1', '4'] - Incorrect (digit 1, digit 2)
meter0984.jpg: Predicted - ['1', '9', '1', '6', '5'], Actual - ['0', '9', '1', '6', '5'] - Incorrect (digit 1)
meter0985.jpg: Predicted - ['0', '0', '1', '9', '6'], Actual - ['0', '0', '1', '9', '4'] - Incorrect (digit 5)
meter0986.jpg: Predicted - ['0', '6', '2', '3', '3'], Actual - ['0', '6', '7', '3', '3'] - Incorrect (digit 3)
meter0987.jpg: Predicted - ['2', '2', '2', '3', '6'], Actual - ['2', '2', '2', '3', '6'] - Correct
meter0988.jpg: Predicted - ['1', '7', '1', '1', '9'], Actual - ['1', '7', '1', '7', '8'] - Incorrect (digit 4, digit 5)
meter0989.jpg: Predicted - ['0', '5', '3', '4', '8'], Actual - ['0', '5', '3', '4', '6'] - Incorrect (digit 5)
meter0990.jpg: Predicted - ['1', '4', '6', '6', '2'], Actual - ['1', '4', '6', '6', '2'] - Correct
meter0991.jpg: Predicted - ['1', '4', '3', '7', '4'], Actual - ['1', '4', '3', '7', '4'] - Correct
meter0992.jpg: Predicted - ['2', '4', '9', '5', '1'], Actual - ['2', '4', '9', '5', '1'] - Correct
meter0993.jpg: Predicted - ['1', '0', '3', '0', '3'], Actual - ['1', '0', '3', '0', '3'] - Correct
meter0994.jpg: Predicted - ['1', '3', '0', '7', '8'], Actual - ['1', '3', '1', '7', '8'] - Incorrect (digit 3)
meter0995.jpg: Predicted - ['0', '0', '0', '8', '4'], Actual - ['4', '0', '0', '8', '0'] - Incorrect (digit 1, digit 5)
meter0996.jpg: Predicted - ['0', '7', '1', '9', '6'], Actual - ['0', '1', '1', '9', '5'] - Incorrect (digit 2, digit 5)
meter0997.jpg: Predicted - ['0', '5', '7', '2', '3'], Actual - ['0', '5', '7', '3', '3'] - Incorrect (digit 4)
meter0998.jpg: Predicted - ['0', '4', '0', '5', '9'], Actual - ['0', '2', '0', '5', '3'] - Incorrect (digit 2, digit 5)
meter0999.jpg: Predicted - ['1', '7', '5', '5', '3'], Actual - ['1', '1', '5', '5', '3'] - Incorrect (digit 2)
meter1000.jpg: Predicted - ['0', '9', '6', '9', '1'], Actual - ['0', '9', '6', '9', '1'] - Correct
meter1001.jpg: Predicted - ['0', '4', '1', '4', '6'], Actual - ['0', '5', '2', '4', '6'] - Incorrect (digit 2, digit 3)
meter1002.jpg: Predicted - ['0', '5', '3', '8', '1'], Actual - ['0', '5', '3', '8', '2'] - Incorrect (digit 5)
meter1003.jpg: Predicted - ['0', '4', '2', '4', '0'], Actual - ['0', '5', '2', '4', '6'] - Incorrect (digit 2, digit 5)
meter1004.jpg: Predicted - ['1', '0', '2', '1', '3'], Actual - ['1', '0', '2', '1', '3'] - Correct
meter1005.jpg: Predicted - ['0', '9', '6', '9', '9'], Actual - ['0', '9', '6', '9', '9'] - Correct
meter1006.jpg: Predicted - ['0', '9', '4', '4', '4'], Actual - ['0', '9', '4', '4', '4'] - Correct
meter1007.jpg: Predicted - ['0', '9', '0', '7', '2'], Actual - ['0', '9', '0', '7', '2'] - Correct
meter1008.jpg: Predicted - ['1', '1', '9', '8', '2'], Actual - ['1', '1', '9', '8', '2'] - Correct
meter1009.jpg: Predicted - ['1', '3', '8', '4', '7'], Actual - ['1', '3', '8', '4', '7'] - Correct
meter1010.jpg: Predicted - ['1', '0', '3', '1', '6'], Actual - ['1', '0', '3', '1', '7'] - Incorrect (digit 5)
meter1011.jpg: Predicted - ['1', '5', '1', '3', '4'], Actual - ['2', '2', '1', '3', '4'] - Incorrect (digit 1, digit 2)
meter1012.jpg: Predicted - ['2', '2', '1', '9', '5'], Actual - ['2', '2', '7', '9', '5'] - Incorrect (digit 3)
meter1013.jpg: Predicted - ['1', '1', '4', '6', '1'], Actual - ['3', '1', '4', '5', '1'] - Incorrect (digit 1, digit 4)
meter1014.jpg: Predicted - ['1', '4', '5', '3', '1'], Actual - ['1', '4', '5', '3', '1'] - Correct
meter1015.jpg: Predicted - ['1', '8', '8', '5', '0'], Actual - ['1', '8', '8', '5', '8'] - Incorrect (digit 5)
meter1016.jpg: Predicted - ['1', '0', '1', '2', '8'], Actual - ['1', '0', '1', '3', '9'] - Incorrect (digit 4, digit 5)
meter1017.jpg: Predicted - ['0', '8', '2', '8', '9'], Actual - ['0', '8', '2', '0', '9'] - Incorrect (digit 4)
meter1018.jpg: Predicted - ['4', '7', '6', '1', '2'], Actual - ['4', '7', '6', '0', '5'] - Incorrect (digit 4, digit 5)
meter1019.jpg: Predicted - ['6', '0', '5', '9', '2'], Actual - ['6', '0', '5', '9', '2'] - Correct
meter1020.jpg: Predicted - ['1', '1', '8', '4', '7'], Actual - ['3', '1', '6', '5', '7'] - Incorrect (digit 1, digit 3, digit 4)
meter1021.jpg: Predicted - ['0', '7', '0', '5', '8'], Actual - ['8', '7', '0', '5', '0'] - Incorrect (digit 1, digit 5)
meter1022.jpg: Predicted - ['2', '3', '3', '2', '2'], Actual - ['2', '3', '3', '2', '2'] - Correct
meter1023.jpg: Predicted - ['0', '7', '8', '6', '7'], Actual - ['0', '7', '8', '6', '7'] - Correct
meter1024.jpg: Predicted - ['0', '3', '3', '4', '8'], Actual - ['0', '3', '3', '8', '6'] - Incorrect (digit 4, digit 5)
meter1025.jpg: Predicted - ['1', '0', '9', '4', '6'], Actual - ['1', '0', '9', '4', '6'] - Correct
meter1026.jpg: Predicted - ['1', '9', '9', '2', '0'], Actual - ['1', '9', '9', '2', '0'] - Correct
meter1027.jpg: Predicted - ['0', '6', '0', '6', '0'], Actual - ['0', '6', '0', '6', '0'] - Correct
meter1028.jpg: Predicted - ['1', '4', '7', '0', '6'], Actual - ['1', '4', '2', '0', '6'] - Incorrect (digit 3)
meter1029.jpg: Predicted - ['0', '9', '2', '8', '3'], Actual - ['0', '3', '3', '8', '3'] - Incorrect (digit 2, digit 3)
meter1030.jpg: Predicted - ['0', '6', '1', '9', '2'], Actual - ['0', '6', '1', '9', '7'] - Incorrect (digit 5)
meter1031.jpg: Predicted - ['0', '6', '5', '6', '9'], Actual - ['0', '6', '5', '6', '9'] - Correct
meter1032.jpg: Predicted - ['1', '4', '2', '7', '9'], Actual - ['1', '4', '2', '7', '9'] - Correct
meter1033.jpg: Predicted - ['1', '0', '9', '2', '2'], Actual - ['1', '0', '9', '2', '2'] - Correct
meter1034.jpg: Predicted - ['0', '7', '7', '7', '5'], Actual - ['1', '1', '7', '7', '5'] - Incorrect (digit 1, digit 2)
meter1035.jpg: Predicted - ['1', '4', '3', '7', '5'], Actual - ['1', '4', '3', '7', '5'] - Correct
meter1036.jpg: Predicted - ['0', '0', '9', '3', '5'], Actual - ['2', '0', '9', '3', '5'] - Incorrect (digit 1)
meter1037.jpg: Predicted - ['0', '7', '9', '5', '6'], Actual - ['0', '7', '0', '5', '1'] - Incorrect (digit 3, digit 5)
meter1038.jpg: Predicted - ['1', '7', '4', '6', '1'], Actual - ['1', '7', '4', '6', '1'] - Correct
meter1039.jpg: Predicted - ['0', '5', '7', '2', '9'], Actual - ['0', '5', '7', '2', '9'] - Correct
meter1040.jpg: Predicted - ['0', '9', '9', '3', '1'], Actual - ['0', '9', '9', '3', '1'] - Correct
meter1041.jpg: Predicted - ['0', '5', '6', '1', '2'], Actual - ['0', '9', '6', '3', '5'] - Incorrect (digit 2, digit 4, digit 5)
meter1042.jpg: Predicted - ['0', '9', '8', '6', '5'], Actual - ['0', '9', '8', '6', '5'] - Correct
meter1043.jpg: Predicted - ['0', '9', '5', '0', '4'], Actual - ['0', '9', '5', '0', '9'] - Incorrect (digit 5)
meter1044.jpg: Predicted - ['0', '8', '9', '7', '9'], Actual - ['0', '8', '9', '7', '9'] - Correct
meter1045.jpg: Predicted - ['1', '0', '0', '2', '6'], Actual - ['0', '8', '0', '2', '6'] - Incorrect (digit 1, digit 2)
meter1046.jpg: Predicted - ['0', '7', '3', '0', '3'], Actual - ['0', '7', '3', '0', '3'] - Correct
meter1047.jpg: Predicted - ['0', '7', '9', '1', '7'], Actual - ['0', '7', '9', '1', '3'] - Incorrect (digit 5)
meter1048.jpg: Predicted - ['1', '1', '7', '1', '9'], Actual - ['1', '1', '7', '1', '9'] - Correct
meter1049.jpg: Predicted - ['1', '3', '4', '7', '4'], Actual - ['1', '3', '4', '7', '4'] - Correct
meter1050.jpg: Predicted - ['1', '0', '5', '4', '6'], Actual - ['1', '8', '5', '4', '6'] - Incorrect (digit 2)
meter1051.jpg: Predicted - ['0', '4', '7', '7', '5'], Actual - ['0', '4', '7', '7', '5'] - Correct
meter1052.jpg: Predicted - ['0', '8', '9', '5', '6'], Actual - ['0', '8', '9', '5', '6'] - Correct
meter1053.jpg: Predicted - ['1', '0', '9', '8', '3'], Actual - ['0', '8', '9', '0', '3'] - Incorrect (digit 1, digit 2, digit 4)
meter1054.jpg: Predicted - ['1', '1', '0', '1', '8'], Actual - ['1', '2', '0', '1', '8'] - Incorrect (digit 2)
meter1055.jpg: Predicted - ['0', '4', '1', '7', '8'], Actual - ['0', '4', '1', '7', '8'] - Correct
meter1056.jpg: Predicted - ['0', '8', '1', '8', '4'], Actual - ['0', '4', '1', '8', '4'] - Incorrect (digit 2)
meter1057.jpg: Predicted - ['0', '3', '1', '9', '4'], Actual - ['0', '3', '9', '3', '1'] - Incorrect (digit 3, digit 4, digit 5)
meter1058.jpg: Predicted - ['1', '4', '4', '4', '3'], Actual - ['1', '4', '4', '4', '3'] - Correct
meter1059.jpg: Predicted - ['1', '0', '0', '9', '3'], Actual - ['1', '0', '0', '9', '3'] - Correct
meter1060.jpg: Predicted - ['0', '3', '3', '0', '2'], Actual - ['0', '3', '3', '0', '2'] - Correct
meter1061.jpg: Predicted - ['0', '0', '5', '3', '8'], Actual - ['0', '0', '5', '3', '8'] - Correct
meter1062.jpg: Predicted - ['0', '8', '6', '4', '5'], Actual - ['0', '8', '6', '4', '5'] - Correct
meter1063.jpg: Predicted - ['0', '5', '6', '5', '4'], Actual - ['0', '5', '6', '5', '4'] - Correct
meter1064.jpg: Predicted - ['1', '3', '3', '7', '5'], Actual - ['1', '3', '3', '7', '5'] - Correct
meter1065.jpg: Predicted - ['1', '2', '3', '2', '7'], Actual - ['1', '2', '3', '2', '6'] - Incorrect (digit 5)
meter1066.jpg: Predicted - ['0', '6', '0', '4', '4'], Actual - ['0', '6', '0', '4', '4'] - Correct
meter1067.jpg: Predicted - ['0', '4', '4', '6', '8'], Actual - ['0', '4', '4', '6', '8'] - Correct
meter1068.jpg: Predicted - ['0', '1', '1', '9', '7'], Actual - ['0', '7', '7', '9', '2'] - Incorrect (digit 2, digit 3, digit 5)
meter1069.jpg: Predicted - ['5', '7', '7', '3', '3'], Actual - ['5', '7', '7', '3', '3'] - Correct
meter1070.jpg: Predicted - ['3', '0', '3', '6', '3'], Actual - ['5', '8', '3', '6', '3'] - Incorrect (digit 1, digit 2)
meter1071.jpg: Predicted - ['2', '0', '3', '2', '5'], Actual - ['2', '0', '3', '2', '5'] - Correct
meter1072.jpg: Predicted - ['0', '8', '8', '5', '4'], Actual - ['0', '8', '8', '5', '4'] - Correct
meter1073.jpg: Predicted - ['0', '4', '1', '5', '7'], Actual - ['0', '4', '1', '5', '7'] - Correct
meter1074.jpg: Predicted - ['0', '3', '4', '5', '6'], Actual - ['0', '3', '6', '5', '6'] - Incorrect (digit 3)
meter1075.jpg: Predicted - ['0', '1', '9', '5', '7'], Actual - ['0', '1', '9', '5', '7'] - Correct
meter1076.jpg: Predicted - ['3', '6', '9', '6', '3'], Actual - ['3', '6', '9', '6', '3'] - Correct
meter1077.jpg: Predicted - ['0', '2', '3', '6', '3'], Actual - ['0', '8', '5', '5', '1'] - Incorrect (digit 2, digit 3, digit 4, digit 5)
meter1078.jpg: Predicted - ['3', '2', '7', '8', '3'], Actual - ['3', '2', '7', '8', '3'] - Correct
meter1079.jpg: Predicted - ['0', '6', '0', '0', '5'], Actual - ['0', '6', '0', '0', '5'] - Correct
meter1080.jpg: Predicted - ['2', '1', '7', '0', '2'], Actual - ['2', '1', '7', '0', '2'] - Correct
meter1081.jpg: Predicted - ['1', '3', '1', '0', '6'], Actual - ['1', '4', '2', '0', '6'] - Incorrect (digit 2, digit 3)
meter1082.jpg: Predicted - ['1', '6', '8', '1', '6'], Actual - ['1', '6', '8', '1', '6'] - Correct
meter1083.jpg: Predicted - ['0', '3', '9', '5', '7'], Actual - ['0', '3', '9', '5', '7'] - Correct
meter1084.jpg: Predicted - ['1', '9', '1', '5', '2'], Actual - ['1', '9', '1', '5', '2'] - Correct
meter1085.jpg: Predicted - ['0', '1', '2', '7', '7'], Actual - ['0', '1', '2', '7', '1'] - Incorrect (digit 5)
meter1086.jpg: Predicted - ['0', '7', '7', '4', '6'], Actual - ['0', '7', '7', '4', '6'] - Correct
meter1087.jpg: Predicted - ['0', '8', '6', '7', '3'], Actual - ['0', '8', '6', '7', '3'] - Correct
meter1088.jpg: Predicted - ['1', '3', '6', '6', '2'], Actual - ['1', '3', '6', '5', '2'] - Incorrect (digit 4)
meter1089.jpg: Predicted - ['0', '5', '4', '9', '0'], Actual - ['0', '5', '4', '9', '0'] - Correct
meter1090.jpg: Predicted - ['1', '6', '3', '4', '8'], Actual - ['1', '6', '3', '4', '8'] - Correct
meter1091.jpg: Predicted - ['1', '4', '9', '5', '5'], Actual - ['1', '4', '9', '5', '5'] - Correct
meter1092.jpg: Predicted - ['0', '8', '8', '1', '6'], Actual - ['0', '8', '8', '1', '6'] - Correct
meter1093.jpg: Predicted - ['0', '7', '3', '6', '3'], Actual - ['0', '7', '3', '6', '3'] - Correct
meter1094.jpg: Predicted - ['0', '9', '5', '0', '9'], Actual - ['0', '9', '5', '0', '9'] - Correct
meter1095.jpg: Predicted - ['1', '2', '5', '2', '7'], Actual - ['1', '2', '5', '2', '4'] - Incorrect (digit 5)
meter1096.jpg: Predicted - ['1', '0', '0', '6', '8'], Actual - ['1', '3', '0', '6', '8'] - Incorrect (digit 2)
meter1097.jpg: Predicted - ['0', '8', '9', '5', '6'], Actual - ['0', '8', '9', '5', '6'] - Correct
meter1098.jpg: Predicted - ['0', '6', '3', '2', '8'], Actual - ['0', '6', '3', '2', '8'] - Correct
meter1099.jpg: Predicted - ['0', '0', '0', '7', '9'], Actual - ['0', '8', '0', '7', '9'] - Incorrect (digit 2)
meter1100.jpg: Predicted - ['1', '0', '3', '1', '6'], Actual - ['1', '0', '3', '1', '7'] - Incorrect (digit 5)
meter1101.jpg: Predicted - ['2', '4', '2', '1', '7'], Actual - ['2', '4', '2', '1', '7'] - Correct
meter1102.jpg: Predicted - ['1', '7', '1', '5', '9'], Actual - ['1', '7', '1', '5', '9'] - Correct
meter1103.jpg: Predicted - ['2', '5', '8', '9', '9'], Actual - ['2', '5', '8', '9', '9'] - Correct
meter1104.jpg: Predicted - ['0', '8', '0', '9', '6'], Actual - ['0', '8', '0', '9', '6'] - Correct
meter1105.jpg: Predicted - ['0', '5', '4', '4', '4'], Actual - ['0', '5', '4', '4', '4'] - Correct
meter1106.jpg: Predicted - ['2', '2', '7', '3', '6'], Actual - ['2', '2', '2', '3', '6'] - Incorrect (digit 3)
meter1107.jpg: Predicted - ['2', '9', '7', '2', '1'], Actual - ['2', '9', '7', '2', '1'] - Correct
meter1108.jpg: Predicted - ['0', '2', '1', '3', '8'], Actual - ['9', '2', '1', '3', '4'] - Incorrect (digit 1, digit 5)
meter1109.jpg: Predicted - ['2', '1', '7', '6', '5'], Actual - ['2', '1', '7', '6', '5'] - Correct
meter1110.jpg: Predicted - ['1', '2', '3', '6', '7'], Actual - ['1', '2', '3', '6', '1'] - Incorrect (digit 5)
meter1111.jpg: Predicted - ['2', '7', '6', '0', '3'], Actual - ['2', '7', '6', '0', '3'] - Correct
meter1112.jpg: Predicted - ['0', '4', '5', '9', '9'], Actual - ['6', '4', '5', '9', '4'] - Incorrect (digit 1, digit 5)
meter1113.jpg: Predicted - ['1', '4', '9', '8', '7'], Actual - ['1', '4', '9', '3', '7'] - Incorrect (digit 4)
meter1114.jpg: Predicted - ['0', '5', '4', '5', '5'], Actual - ['0', '5', '4', '5', '5'] - Correct
meter1115.jpg: Predicted - ['0', '3', '4', '1', '1'], Actual - ['0', '3', '4', '1', '1'] - Correct
meter1116.jpg: Predicted - ['2', '6', '4', '4', '5'], Actual - ['2', '6', '4', '4', '5'] - Correct
meter1117.jpg: Predicted - ['0', '1', '9', '3', '5'], Actual - ['0', '1', '9', '3', '5'] - Correct
meter1118.jpg: Predicted - ['1', '2', '2', '7', '0'], Actual - ['1', '2', '2', '7', '0'] - Correct
meter1119.jpg: Predicted - ['0', '0', '7', '0', '6'], Actual - ['0', '0', '7', '0', '6'] - Correct
meter1120.jpg: Predicted - ['0', '5', '5', '2', '8'], Actual - ['0', '5', '5', '2', '8'] - Correct
meter1121.jpg: Predicted - ['0', '3', '5', '4', '8'], Actual - ['0', '3', '5', '4', '6'] - Incorrect (digit 5)
meter1122.jpg: Predicted - ['1', '4', '6', '0', '7'], Actual - ['1', '4', '6', '0', '7'] - Correct
meter1123.jpg: Predicted - ['2', '2', '7', '5', '7'], Actual - ['2', '2', '7', '5', '7'] - Correct
meter1124.jpg: Predicted - ['0', '3', '2', '8', '8'], Actual - ['0', '3', '2', '8', '8'] - Correct
meter1125.jpg: Predicted - ['1', '8', '5', '8', '8'], Actual - ['4', '4', '5', '8', '9'] - Incorrect (digit 1, digit 2, digit 5)
meter1126.jpg: Predicted - ['2', '7', '8', '4', '1'], Actual - ['2', '7', '6', '4', '1'] - Incorrect (digit 3)
meter1127.jpg: Predicted - ['0', '5', '2', '4', '2'], Actual - ['0', '5', '2', '4', '2'] - Correct
meter1128.jpg: Predicted - ['0', '0', '3', '9', '1'], Actual - ['1', '0', '3', '9', '1'] - Incorrect (digit 1)
meter1129.jpg: Predicted - ['4', '3', '8', '3', '3'], Actual - ['4', '3', '8', '3', '3'] - Correct
meter1130.jpg: Predicted - ['0', '1', '1', '2', '5'], Actual - ['0', '1', '1', '2', '5'] - Correct
meter1131.jpg: Predicted - ['3', '3', '1', '9', '0'], Actual - ['3', '3', '1', '9', '0'] - Correct
meter1132.jpg: Predicted - ['2', '1', '1', '2', '1'], Actual - ['1', '1', '1', '2', '1'] - Incorrect (digit 1)
meter1133.jpg: Predicted - ['0', '0', '5', '9', '8'], Actual - ['8', '9', '5', '9', '8'] - Incorrect (digit 1, digit 2)
meter1134.jpg: Predicted - ['0', '3', '8', '6', '8'], Actual - ['0', '3', '8', '6', '8'] - Correct
meter1135.jpg: Predicted - ['0', '4', '9', '2', '6'], Actual - ['0', '4', '9', '2', '6'] - Correct
meter1136.jpg: Predicted - ['2', '0', '3', '7', '1'], Actual - ['1', '8', '3', '7', '1'] - Incorrect (digit 1, digit 2)
meter1137.jpg: Predicted - ['1', '6', '5', '9', '0'], Actual - ['1', '6', '5', '9', '0'] - Correct
meter1138.jpg: Predicted - ['0', '5', '8', '3', '8'], Actual - ['9', '5', '8', '3', '0'] - Incorrect (digit 1, digit 5)
meter1139.jpg: Predicted - ['2', '4', '2', '1', '2'], Actual - ['2', '4', '2', '1', '2'] - Correct
meter1140.jpg: Predicted - ['5', '0', '6', '8', '9'], Actual - ['5', '0', '6', '8', '9'] - Correct
meter1141.jpg: Predicted - ['0', '2', '2', '4', '8'], Actual - ['4', '2', '2', '4', '0'] - Incorrect (digit 1, digit 5)
meter1142.jpg: Predicted - ['0', '9', '7', '2', '6'], Actual - ['6', '9', '7', '2', '6'] - Incorrect (digit 1)
meter1143.jpg: Predicted - ['4', '1', '8', '8', '4'], Actual - ['4', '8', '8', '1', '0'] - Incorrect (digit 2, digit 4, digit 5)
meter1144.jpg: Predicted - ['0', '3', '1', '4', '4'], Actual - ['0', '3', '1', '4', '4'] - Correct
meter1145.jpg: Predicted - ['0', '0', '1', '6', '6'], Actual - ['0', '8', '1', '0', '6'] - Incorrect (digit 2, digit 4)
meter1146.jpg: Predicted - ['1', '8', '5', '9', '5'], Actual - ['7', '8', '5', '9', '5'] - Incorrect (digit 1)
meter1147.jpg: Predicted - ['2', '1', '1', '2', '1'], Actual - ['2', '1', '1', '2', '1'] - Correct
meter1148.jpg: Predicted - ['0', '6', '1', '5', '6'], Actual - ['6', '6', '1', '5', '1'] - Incorrect (digit 1, digit 5)
meter1149.jpg: Predicted - ['1', '2', '3', '2', '5'], Actual - ['1', '2', '3', '2', '5'] - Correct
meter1150.jpg: Predicted - ['0', '3', '9', '8', '6'], Actual - ['0', '3', '9', '8', '6'] - Correct
meter1151.jpg: Predicted - ['4', '8', '1', '4', '9'], Actual - ['4', '8', '1', '4', '9'] - Correct
meter1152.jpg: Predicted - ['0', '8', '9', '1', '8'], Actual - ['0', '6', '9', '1', '8'] - Incorrect (digit 2)
meter1153.jpg: Predicted - ['0', '0', '2', '7', '1'], Actual - ['7', '5', '2', '7', '0'] - Incorrect (digit 1, digit 2, digit 5)
meter1154.jpg: Predicted - ['0', '0', '8', '8', '1'], Actual - ['1', '6', '8', '0', '0'] - Incorrect (digit 1, digit 2, digit 4, digit 5)
meter1155.jpg: Predicted - ['3', '9', '9', '4', '8'], Actual - ['9', '9', '9', '4', '0'] - Incorrect (digit 1, digit 5)
meter1156.jpg: Predicted - ['0', '7', '5', '1', '1'], Actual - ['0', '7', '5', '1', '1'] - Correct
meter1157.jpg: Predicted - ['1', '0', '0', '9', '1'], Actual - ['6', '0', '0', '9', '1'] - Incorrect (digit 1)
meter1158.jpg: Predicted - ['1', '5', '6', '0', '9'], Actual - ['4', '5', '6', '0', '9'] - Incorrect (digit 1)
meter1159.jpg: Predicted - ['1', '2', '6', '3', '0'], Actual - ['1', '2', '6', '3', '0'] - Correct
meter1160.jpg: Predicted - ['6', '7', '5', '4', '8'], Actual - ['6', '1', '5', '4', '8'] - Incorrect (digit 2)
meter1161.jpg: Predicted - ['3', '4', '1', '7', '4'], Actual - ['3', '4', '1', '7', '6'] - Incorrect (digit 5)
meter1162.jpg: Predicted - ['0', '4', '9', '3', '8'], Actual - ['0', '4', '9', '3', '8'] - Correct
meter1163.jpg: Predicted - ['0', '6', '1', '5', '8'], Actual - ['0', '6', '1', '5', '8'] - Correct
meter1164.jpg: Predicted - ['1', '3', '4', '9', '0'], Actual - ['1', '3', '4', '9', '0'] - Correct
meter1165.jpg: Predicted - ['1', '7', '2', '2', '1'], Actual - ['1', '7', '2', '2', '7'] - Incorrect (digit 5)
meter1166.jpg: Predicted - ['0', '9', '9', '3', '1'], Actual - ['0', '9', '9', '3', '1'] - Correct
meter1167.jpg: Predicted - ['0', '6', '9', '4', '2'], Actual - ['0', '6', '9', '4', '2'] - Correct
meter1168.jpg: Predicted - ['0', '8', '4', '1', '2'], Actual - ['0', '8', '4', '1', '2'] - Correct
meter1169.jpg: Predicted - ['0', '5', '7', '2', '9'], Actual - ['0', '5', '7', '2', '9'] - Correct
meter1170.jpg: Predicted - ['1', '2', '3', '5', '3'], Actual - ['1', '2', '3', '5', '3'] - Correct
meter1171.jpg: Predicted - ['0', '4', '4', '2', '5'], Actual - ['0', '4', '4', '2', '5'] - Correct
meter1172.jpg: Predicted - ['1', '0', '5', '4', '7'], Actual - ['1', '0', '5', '4', '7'] - Correct
meter1173.jpg: Predicted - ['0', '7', '0', '5', '1'], Actual - ['0', '7', '0', '5', '1'] - Correct
meter1174.jpg: Predicted - ['0', '6', '2', '1', '0'], Actual - ['0', '6', '2', '1', '0'] - Correct
meter1175.jpg: Predicted - ['1', '1', '4', '9', '8'], Actual - ['1', '1', '4', '9', '6'] - Incorrect (digit 5)
meter1176.jpg: Predicted - ['0', '8', '0', '2', '6'], Actual - ['0', '8', '0', '2', '6'] - Correct
meter1177.jpg: Predicted - ['1', '0', '0', '9', '1'], Actual - ['1', '0', '0', '9', '3'] - Incorrect (digit 5)
meter1178.jpg: Predicted - ['1', '4', '4', '2', '1'], Actual - ['1', '4', '4', '2', '5'] - Incorrect (digit 5)
meter1179.jpg: Predicted - ['1', '2', '8', '3', '7'], Actual - ['1', '2', '8', '3', '7'] - Correct
meter1180.jpg: Predicted - ['1', '0', '4', '1', '2'], Actual - ['1', '0', '4', '1', '2'] - Correct
meter1181.jpg: Predicted - ['0', '8', '7', '6', '4'], Actual - ['0', '8', '7', '6', '4'] - Correct
meter1182.jpg: Predicted - ['1', '3', '0', '0', '1'], Actual - ['1', '4', '0', '0', '1'] - Incorrect (digit 2)
meter1183.jpg: Predicted - ['0', '8', '9', '8', '3'], Actual - ['0', '8', '9', '0', '3'] - Incorrect (digit 4)
meter1184.jpg: Predicted - ['0', '4', '1', '7', '8'], Actual - ['0', '4', '1', '7', '8'] - Correct
meter1185.jpg: Predicted - ['1', '4', '9', '7', '9'], Actual - ['1', '4', '9', '7', '9'] - Correct
meter1186.jpg: Predicted - ['0', '0', '0', '0', '0'], Actual - ['0', '0', '0', '0', '0'] - Correct
meter1187.jpg: Predicted - ['0', '9', '8', '3', '9'], Actual - ['0', '9', '8', '3', '9'] - Correct
meter1188.jpg: Predicted - ['0', '2', '1', '9', '1'], Actual - ['0', '2', '7', '9', '1'] - Incorrect (digit 3)
meter1189.jpg: Predicted - ['0', '3', '2', '2', '1'], Actual - ['0', '3', '2', '2', '3'] - Incorrect (digit 5)
meter1190.jpg: Predicted - ['0', '8', '7', '8', '8'], Actual - ['0', '1', '7', '8', '8'] - Incorrect (digit 2)
meter1191.jpg: Predicted - ['2', '0', '4', '0', '8'], Actual - ['2', '0', '4', '6', '9'] - Incorrect (digit 4, digit 5)
meter1192.jpg: Predicted - ['1', '3', '0', '0', '4'], Actual - ['1', '3', '0', '0', '4'] - Correct
meter1193.jpg: Predicted - ['0', '6', '0', '1', '8'], Actual - ['0', '6', '0', '1', '8'] - Correct
meter1194.jpg: Predicted - ['1', '8', '3', '9', '5'], Actual - ['1', '8', '3', '9', '5'] - Correct
meter1195.jpg: Predicted - ['0', '5', '5', '2', '5'], Actual - ['0', '5', '5', '2', '5'] - Correct
meter1196.jpg: Predicted - ['3', '7', '1', '7', '1'], Actual - ['3', '7', '1', '7', '0'] - Incorrect (digit 5)
meter1197.jpg: Predicted - ['0', '6', '6', '7', '3'], Actual - ['0', '6', '6', '7', '3'] - Correct
meter1198.jpg: Predicted - ['0', '1', '5', '2', '1'], Actual - ['0', '1', '5', '2', '1'] - Correct
meter1199.jpg: Predicted - ['2', '2', '2', '4', '7'], Actual - ['2', '2', '2', '4', '7'] - Correct
meter1200.jpg: Predicted - ['0', '7', '3', '1', '7'], Actual - ['0', '7', '3', '1', '7'] - Correct

Total correct predictions: 146
Accuracy: 58.40%

Total correct digits: 1089
Overall digit accuracy: 87.12%
Digit 1 accuracy: 87.20%
Digit 2 accuracy: 86.80%
Digit 3 accuracy: 90.40%
Digit 4 accuracy: 91.20%
Digit 5 accuracy: 80.00%
Total digit accuracy across all readings: 87.12%


In den beiden Projektschritten  1. Counter Detection (YOLOv5) und 2. Digit Detection (CR-NET) wurde die Wirksamkeit eines zweistufigen Ansatzes für die automatische Zählerablesung (AMR) nachgewiesen.

In dem kombinierten Prozess erreichte das CR-NET-Modell eine Gesamtablesegenauigkeit von 58,40 % bei 250 Testbildern, mit einer hohen Ziffernerkennungsgenauigkeit von 87,12 %. Das legt nahe, dass ein zweistufiger Ansatz, beginnend mit einer robusten Zählererkennung mit Modellen wie YOLOv5, gefolgt von einer präzisen Ziffernerkennung mit CR-NET, sehr effektiv ist. Um zuverlässigere Ergebnisse zu erzielen, sind jedoch größere Rechenressourcen und umfangreichere Trainingsdatensätze erforderlich (Epochen >50) Außerdem ist das derzeitige Modell, das auf fünfstelligen Zählern trainiert wurde, für reale Anwendungen, bei denen eine größere Vielfalt von Zählertypen und Ziffernzahlen üblich ist, möglicherweise noch nicht praktikabel. Eine weitere Verfeinerung und Skalierung des Modells ist notwendig, um seine praktische Anwendbarkeit zu verbessern.